# Подготовка пайплайна тестирования

Установка пакета с алгоритмами

In [8]:
from numba import short
!pip install cp2025

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 13.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib_metadata 8.7.0
    Uninstalling importlib_metadata-8.7.0:
      Successfully uninstalled importlib_metadata-8.7.0
  Attempting uninst

Импорт зависимостей

In [1]:
import numpy as np
import pandas as pd

from cp2025.algorithms.AggregationKAnonymityTimeOptimal import AggregationKAnonymityTimeOptimal
from cp2025.algorithms.AggregationLDiversityTimeOptimal import AggregationLDiversityTimeOptimal
from cp2025.algorithms.Datafly import Datafly
from cp2025.algorithms.GeneralizationGreedyByOneEqualSizedGroups import GeneralizationGreedyByOneEqualSizedGroups
from cp2025.algorithms.GeneralizationKAnonymityTimeOptimal import GeneralizationKAnonymityTimeOptimal
from cp2025.algorithms.GeneralizationLDiversityTimeOptimal import GeneralizationLDiversityTimeOptimal
from cp2025.algorithms.groupjoin import *
from cp2025.algorithms.IdentifierHasher import IdentifierHasher
from cp2025.algorithms.RandomizationDepersonalizator import RandomizationBaselineDepersonalizator
from cp2025.algorithms.Shuffler import Shuffler
from cp2025.algorithms.ShufflerInBatches import ShufflerInBatches
from cp2025.algorithms.SuppressionKAnonymityBaseline import SuppressionKAnonymityBaseline
from cp2025.algorithms.SuppressionKAnonymityTimeOptimal import SuppressionKAnonymityTimeOptimal
from cp2025.algorithms.SuppressionLDiversityBaseline import SuppressionLDiversityBaseline
from cp2025.algorithms.SuppressionLDiversityTimeOptimal import SuppressionLDiversityTimeOptimal
from cp2025.algorithms.SuppressionTClosenessBaseline import SuppressionTClosenessBaseline
from cp2025.algorithms.DistributedDataKAnonymityDepersonalization import DistributedDataOwnerKAnonymityDepersonalizator
from cp2025.utility.metrics import *
from cp2025.utility.boosting_security_score import get_boosting_security_score
from sklearn.model_selection import train_test_split
from cp2025.utility.prepare_data import prepare_data
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, mean_absolute_percentage_error
import time

Функция оценки качества обезличивания при помощи следующих мертик:

Для полезности обезличенных данных:

* средний размер класса эквивалентности
* отношение количества уникальных записей к общему числу записей (distinctness)
* доля изменившихся отдельных значений в датасете
* неоднородная энтропия (non-uniform entropy)
* потеря информации на основе энтропии (entropy-based information loss)
* оценка расстояния между исходными и получившимися данными поэлементно (my_by_element_distance)
* решение задач классификации или регрессии при помощи градиентного бустинга на изначальном и обезличенном датасетах и сравнение качества работы моделей


Для рисков деобезличивания:

* оценка максимального k, при котором выполняется k-анонимность
* оценка максимального l, при котором выполняется l-разнообразие
* оценка минимального t, при котором выполняется t-близость
* adversarial knowledge gain, описанный в пункте 4.4 статьи https://desfontain.es/PDFs/PhD/TheCostOfPrivacyDestructionOfDataMiningUtilityInAnonymizedDataPublishing.pdf
* adversarial accuracy gain, описанный в пункте 4.4 статьи https://desfontain.es/PDFs/PhD/TheCostOfPrivacyDestructionOfDataMiningUtilityInAnonymizedDataPublishing.pdf
* оценка accuracy деобезличивания при помощи градиентного бустинга


In [201]:
import warnings
from sklearn.exceptions import ConvergenceWarning

def score_depersonalization_quality(initial_df, depersonalized_df, train, test, train_depersonalized, test_depersonalized, quasi_identifiers_columns_ids, quasi_identifiers_columns_types,sensetive_columns_ids, sensetive_columns_types, predicted_column,quasi_identifiers_boosting_ids, sensitive_boosting_ids, generalization = False, short = False,classification=True):
    global seed

    # depersonalization risk
    k = find_k_anonimus(depersonalized_df[:,quasi_identifiers_columns_ids])
    print(f"Максимальное k: {k}")
    l = find_l_diverse(depersonalized_df[:,quasi_identifiers_columns_ids], depersonalized_df[:,sensetive_columns_ids])
    print(f"Максимальное l: {l}")
    t = find_t_close(depersonalized_df[:,quasi_identifiers_columns_ids], depersonalized_df[:,sensetive_columns_ids], sensetive_columns_types) # TODO: fix to work with many sensetives
    print(f"Минимальное t: {t}")
    adversarial_knowledges = []
    for i in sensetive_columns_ids:
        adversarial_knowledges.append(adversarial_knowledge_gain(depersonalized_df[:,quasi_identifiers_columns_ids], depersonalized_df[:,i]))
    print(f"Adversarial knowledge gain: {max(adversarial_knowledges)}")
    adversarial_accuracy_gains = []
    for i in sensetive_columns_ids:
        adversarial_accuracy_gains.append(adversarial_accuracy_gain(depersonalized_df[:,quasi_identifiers_columns_ids], depersonalized_df[:,i]))
    print(f"Adversarial accuracy gain: {max(adversarial_accuracy_gains)}")
    risks = [k, l, t, max(adversarial_knowledges), max(adversarial_accuracy_gains)]

    # security boosting
    if not short:
        quasi_identifiers_columns_types_boosting = []
        sensitive_columns_types_boosting = []
        for i in quasi_identifiers_boosting_ids:
            quasi_identifiers_columns_types_boosting.append(quasi_identifiers_columns_types[quasi_identifiers_columns_ids.index(i)])
        for i in sensitive_boosting_ids:
            sensitive_columns_types_boosting.append(sensetive_columns_types[sensetive_columns_ids.index(i)])
        train_initial, test_initial = prepare_data(train[:,quasi_identifiers_boosting_ids], test[:,quasi_identifiers_boosting_ids], [0] * len(quasi_identifiers_boosting_ids) , quasi_identifiers_columns_types_boosting, normalize=True)
        train_depersonalized_sec, test_depersonalized_sec = prepare_data(train_depersonalized[:,quasi_identifiers_boosting_ids + sensitive_boosting_ids], test_depersonalized[:,quasi_identifiers_boosting_ids + sensitive_boosting_ids], [generalization] * len(quasi_identifiers_boosting_ids) + [0]*len(sensitive_boosting_ids), quasi_identifiers_columns_types_boosting+sensitive_columns_types_boosting, normalize=True)

    # utility
    average_equivalence_class = average_equivalence_class_size(depersonalized_df[:,quasi_identifiers_columns_ids])
    print(f"Средний размер класса эквивалентности: {average_equivalence_class}")
    distinctness_score = distinctness(depersonalized_df[:,quasi_identifiers_columns_ids])
    print(f"Distinctness: {distinctness_score}")
    changed_proportion_score = changed_proportion(initial_df[:,quasi_identifiers_columns_ids], depersonalized_df[:,quasi_identifiers_columns_ids])
    print(f"Доля изменённых значений: {changed_proportion_score}")
    non_uniform_entropy_score = non_uniform_entropy(depersonalized_df[:,quasi_identifiers_columns_ids])
    print(f"Неоднородная энтропия: {non_uniform_entropy_score}")
    entropy_based_information_loss_score = entropy_based_information_loss(initial_df[:,quasi_identifiers_columns_ids], depersonalized_df[:,quasi_identifiers_columns_ids])
    print(f"Потеря информации на основе энтропии {entropy_based_information_loss_score}")
    my_by_element_distance_score = my_by_element_distance(initial_df[:,quasi_identifiers_columns_ids], depersonalized_df[:,quasi_identifiers_columns_ids], quasi_identifiers_columns_types)
    print(f"My by element distance: {my_by_element_distance_score}")

    # utility boosting
    utility_boosting_score = None
    if not short:
        parameters = {
            #'n_estimators':list(range(10, 201, 15)),
            #'alpha':[i / 10 for i in range(1, 10, 4)],
            'min_samples_leaf':[i for i in range(1, 6)],
            'criterion':['squared_error'],
        }
        quasi_identifiers_columns_types_boosting = quasi_identifiers_columns_types_boosting.copy()
        quasi_identifiers_boosting_ids = quasi_identifiers_boosting_ids.copy()
        sensitive_columns_types_boosting = sensitive_columns_types_boosting.copy()
        sensitive_boosting_ids = sensitive_boosting_ids.copy()
        if predicted_column in quasi_identifiers_boosting_ids:
            quasi_identifiers_columns_types_boosting.pop(quasi_identifiers_boosting_ids.index(predicted_column))
            quasi_identifiers_boosting_ids.remove(predicted_column)
        if predicted_column in sensitive_boosting_ids:
            sensitive_columns_types_boosting.pop(sensitive_boosting_ids.index(predicted_column))
            sensitive_boosting_ids.remove(predicted_column)
        train_depersonalized, test_depersonalized = prepare_data(train_depersonalized[:,quasi_identifiers_boosting_ids + sensitive_boosting_ids], test_depersonalized[:,quasi_identifiers_boosting_ids + sensitive_boosting_ids], [generalization] * len(quasi_identifiers_boosting_ids) + [0]*len(sensitive_boosting_ids), quasi_identifiers_columns_types_boosting+sensitive_columns_types_boosting, normalize=True)
        y_train = train[:,predicted_column]
        y_test = test[:,predicted_column]
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=ConvergenceWarning)
            if classification:
                reg = GridSearchCV(GradientBoostingClassifier(random_state=seed,n_estimators=200), parameters)
                reg.fit(train_depersonalized, y_train)
                res = reg.predict(test_depersonalized)
                #utility_boosting_score = ((res - y_test)**2).mean()**0.5
                utility_boosting_score = accuracy_score(y_test, res)
            else:
                reg = GridSearchCV(GradientBoostingRegressor(random_state=seed,n_estimators=200), parameters)
                reg.fit(train_depersonalized, y_train)
                res = reg.predict(test_depersonalized)
                utility_boosting_score = mean_absolute_percentage_error(y_test, res)#((res - y_test)**2).mean()**0.5
            print(f"Utility boosting score: {utility_boosting_score}")
            
    utilities = [average_equivalence_class, distinctness_score, changed_proportion_score, non_uniform_entropy_score, entropy_based_information_loss_score, my_by_element_distance_score, utility_boosting_score]
    return utilities + risks

Загрузка датасетов:

* df_caggle - датасет с данными по займам с caggle
* df_generated - датасет, сгенерированный ChatGpt (идентификаторы убраны)
* df_short - укороченный до 12 строк и 2 столбцов с квази-идентификаторами df_caggle для baseline алгоритмов (так как задача достижения оптимальной по потере полезности данных k-анонимности - NP-сложная задача)

Описание датасета df_caggle:

* ID - идентификатор записи (идентификатор)
* Age - возраст (квазиидентификатор)
* Experience - стаж (квазиидентификатор)
* Income - заработная плата (чувствительное значение)
* ZIP Code - почтовый индекс (квазиидентификатор)
* Family - число членов семьи (квазиидентификатор)
* CCAvg - средний процент по кредитной карте
* Education - уровень образования (квазиидентификатор)
* Mortgage - размер ипотеки (чувствительное значение)
* Personal Loan - число потребительских кредитов (чувствительное значение)
* Securities Account - наличие счёта ценных бумаг (чувствительное значение)
* CD Account - наличие CD-счёта (чувствительное значение)
* Online - использование онлайн услуг (чувствительное значение)
* CreditCard - наличие кредитной карты (чувствительное значение)


Описание датасета df_generated:

* gender - пол (квазиидентификатор)
* birth_date - дата рождения (квазиидентификатор)
* age - возраст (квазиидентификатор)
* city - город (квазиидентификатор)
* postal_code - почтовый индекс (квазиидентификатор)
* country - страна (квазиидентификатор)
* loan_purpose - цель займа (квазиидентификатор)
* education_level - уровень образования (квазиидентификатор)
* employment_status - занятость (квазиидентификатор)
* years_of_experience - стаж (квазиидентификатор)
* family_size - число членов семьи (квазиидентификатор)
* number_of_children - число детей (квазиидентификатор)
* marital_status - статус брака (квазиидентификатор)
* housing_status - статус жилья (квазиидентификатор)
* industry - сфера деятельности (квазиидентификатор)
* disease - заболевание (чувствительное значение)
* chronic_condition - хроническое заболевание (чувствительное значение)
* insurance_status - страховой статус (чувствительное значение)
* monthly_medical_expenses_usd - траты на лечение в месяц в долларах (чувствительное значение)
* monthly_income_usd - заработная плата в долларах (чувствительное значение)
* monthly_expenses_usd - траты в месяц в долларах (чувствительное значение)
* loan_amount_usd - общий объём займов в долларах (чувствительное значение)
* credit_score - кредитный рейтинг (чувствительное значение)

In [72]:
!wget https://raw.githubusercontent.com/ShompolovMaxim/CP_2025/refs/heads/main/data/Bank_Personal_Loan_Modelling.csv
!wget https://raw.githubusercontent.com/ShompolovMaxim/CP_2025/refs/heads/main/data/generated.csv
df_caggle = pd.read_csv("Bank_Personal_Loan_Modelling.csv", keep_default_na=False)
df_generated = pd.read_csv("generated.csv", keep_default_na=False)
df_short = df_caggle[df_caggle['Mortgage'] != 0][:12]
df_caggle = df_caggle[:1000]
df_generated = df_generated[:1000]

In [73]:
df_caggle

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.60,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.50,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.00,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.70,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.00,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,28,3,45,94305,2,1.60,3,0,0,0,0,1,1
996,997,33,6,49,92037,2,1.67,2,214,0,0,0,0,0
997,998,46,20,69,92780,3,2.10,1,0,0,0,0,0,0
998,999,52,27,94,93106,1,2.80,2,333,0,0,0,1,0


In [74]:
df_generated

,gender,birth_date,age,city,postal_code,country,loan_purpose,education_level,employment_status,years_of_experience,...,housing_status,industry,disease,chronic_condition,insurance_status,monthly_medical_expenses_usd,monthly_income_usd,monthly_expenses_usd,loan_amount_usd,credit_score
0,Female,1991-08-03,34,Bryanside,45471,Trinidad and Tobago,Car,Master's,Employed,1,...,Living with Family,Transportation,None,Acid Reflux,Partially Insured,278.55,2114.35,1386.25,10231.34,666
1,Male,2001-07-10,24,Seanfurt,21310,United Arab Emirates,Business,Associate Degree,Unemployed,0,...,Mortgage,Hospitality,None,None,Uninsured,868.34,1310.32,1113.01,4481.19,783
2,Female,1969-09-28,56,Davebury,879,Trinidad and Tobago,Car,Bachelor's,Student,35,...,Rent,Finance,Diabetes,Seasonal Allergy,Partially Insured,1740.94,20120.36,8986.48,61557.95,641
3,Male,1954-04-08,71,Chadland,22475,Belarus,Business,PhD,Student,28,...,Living with Family,Hospitality,Hypertension,Seasonal Allergy,Partially Insured,406.40,15725.94,14100.05,8575.19,598
4,Male,1948-01-31,77,New Hannah,63590,Trinidad and Tobago,Personal,PhD,Unemployed,51,...,Rent,IT,Hypertension,Seasonal Allergy,Uninsured,138.17,31099.11,21390.76,29449.51,491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Male,1943-09-09,82,East Evelyn,63863,United Arab Emirates,Education,PhD,Unemployed,21,...,Rent,Finance,Depression,Seasonal Allergy,Partially Insured,95.55,14230.14,10762.25,61521.07,347
996,Female,1957-01-09,68,Port Christopherville,14460,Western Sahara,Education,PhD,Self-employed,2,...,Rent,Construction,Depression,Arthritis,Uninsured,785.21,3286.58,2910.79,385.10,742
997,Female,1947-06-11,78,South Tiffany,32757,Western Sahara,Education,PhD,Employed,13,...,Mortgage,Construction,Diabetes,None,Insured,1655.28,8994.30,7238.36,5005.53,537
998,Female,1957-01-25,68,Erichaven,23654,United Arab Emirates,Business,Master's,Self-employed,18,...,Mortgage,Government,None,Arthritis,Insured,972.61,10305.67,9175.70,41670.16,300


In [75]:
df_short = df_short.to_numpy()
df_short = np.delete(df_short, (0,9,10,11,12,13), axis=1)
df_short = np.delete(df_short, (3, 4, 6), axis=1)
df_caggle = df_caggle.to_numpy()
df_caggle = np.delete(df_caggle, (0,9,10,11,12,13), axis=1)
df_generated = df_generated.to_numpy()
df_generated = np.delete(df_generated, (15, 17, 19, 20, 21, 22), axis=1)

seed = 8

train_short, test_short = train_test_split(df_short, test_size=0.2, random_state=seed, shuffle=True)
train_caggle, test_caggle = train_test_split(df_caggle, test_size=0.2, random_state=seed, shuffle=True)
train_generated, test_generated = train_test_split(df_generated, test_size=0.2, random_state=seed, shuffle=True)

Задать параметры датасетов:

* quasi_identifiers_columns_ids - номера столбцов с квазиидентификаторами
* quasi_identifiers_columns_types - типы столбцов с квазиидентификаторами (вещественные, порядковые или номинальные)
* sensetive_columns_ids - номера столбцов с чувствительными значениями
* sensetive_columns_types - типы столбцов с чувствительными значениями (вещественные, порядковые или номинальные)


In [216]:
quasi_identifiers_columns_ids_caggle = [0,1,3,4,6,]
quasi_identifiers_columns_types_caggle = ['real','real','ordered','real','real']
sensetive_columns_ids_caggle = [2,5,7]
sensetive_columns_types_caggle = ['real']*len(sensetive_columns_ids_caggle)
quasi_identifiers_boosting_ids_caggle = [0,1,3,4,6,]
sensitive_boosting_ids_caggle = [2,5,7]

quasi_identifiers_columns_ids_generated = [i for i in range(15)]
quasi_identifiers_columns_types_generated = ['unordered','ordered','real','unordered','unordered','unordered','unordered','unordered','unordered','real','real','real','unordered','unordered','unordered']
sensetive_columns_ids_generated = [i for i in range(15, 17)]
sensetive_columns_types_generated = ['unordered','real']#['unordered','unordered','unordered','real','real','real','real','real']
quasi_identifiers_boosting_ids_generated = [0, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]#[0, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
sensitive_boosting_ids_generated = sensetive_columns_ids_generated

predicted_column_caggle = 4
predicted_column_generated = 16
classification_caggle = True
classification_generated =False

# Эксперименты с алгоритмами обезличивания квазиидентификаторов для общих задач

Задать параметры экспериментов:

* наборы значений (k, l, t) для метрик k-анонимности, l-разнообразия и t-близости (3 массива одинаковой длины, из необходимых массивов выбираются значения с одинаковыми индексами)
* аналогичные наборы значений для короткого датасета
* значения scale для рандомизации
* значение параметра генератора случайных чисел

In [77]:
k_s = [2, 10, 100]
l_s = [2, 3, 4]
t_s = [1, 0.5, 0.1]
k_s_short = [2, 3, 4]
scales = [0.05, 0.25, 1]

Все алгоритмы, рассматриваемые в этом разделе, подавляют все идентификаторы, поэтому для простоты работы идентификаторы заранее убираются из датасетов

Функция для проведения эксперимента над алгоритмом обезличивания с заданным датасетом и его характеристиками

In [81]:
def run_test_on_depersonalizer(df, train, test, dep, quasi_identifiers_columns_ids, quasi_identifiers_columns_types, sensetive_columns_ids, sensetive_columns_types, predicted_column, quasi_identifiers_boosting_ids, sensitive_boosting_ids, generalization=False, short=False, classification = True):
    if short:
        quasi_identifiers_columns_ids = [0,1]
        quasi_identifiers_columns_types = quasi_identifiers_columns_types[:2]
        sensetive_columns_ids = [2,3,4]
    start_time = time.time()
    depersonalized_df, _ = dep.depersonalize(df, identifiers_ids=[], quasi_identifiers_ids=quasi_identifiers_columns_ids, sensitives_ids=sensetive_columns_ids)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Время выполнения: {elapsed_time} секунд")
    depersonalized_train = None
    depersonalized_test = None
    if not short:
        depersonalized_train, _ = dep.depersonalize(train, identifiers_ids=[], quasi_identifiers_ids=quasi_identifiers_columns_ids, sensitives_ids=sensetive_columns_ids)
        depersonalized_test, _ = dep.depersonalize(test, identifiers_ids=[], quasi_identifiers_ids=quasi_identifiers_columns_ids, sensitives_ids=sensetive_columns_ids)
    return score_depersonalization_quality(df,depersonalized_df,train,test,depersonalized_train,depersonalized_test, quasi_identifiers_columns_ids,quasi_identifiers_columns_types,sensetive_columns_ids,sensetive_columns_types,predicted_column,quasi_identifiers_boosting_ids, sensitive_boosting_ids,generalization=generalization, short = short, classification=classification) + [elapsed_time]

In [82]:
columns = ["Средний размер класса эквивалентности", "Distinctness", "Доля изменённых значений", "Неоднородная энтропия", "Потеря информации на основе энтропии", "My by element distance", "Utility boosting score", "max k", "max l", "min t", "Adversarial knowledge gain", "Adversarial accuracy gain", "Время выполнения, секунд"]
index = ['generated', 'caggle', 'short']

## Подсчёт utility boosting score для изначальных датасетов

### Для Kaggle

In [217]:
score_depersonalization_quality(df_caggle,df_caggle,train_caggle,test_caggle,train_caggle,test_caggle, quasi_identifiers_columns_ids_caggle,quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle,predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle,generalization=False, short = False, classification=False)[6]

Максимальное k: 1
Максимальное l: 1
Минимальное t: 0.9115980551052006
Adversarial knowledge gain: 0.9897069999999962
Adversarial accuracy gain: 0.9740000000000009
Средний размер класса эквивалентности: 1.0050251256281406
Distinctness: 0.995
Доля изменённых значений: 0.0
Неоднородная энтропия: 9.955784284662084
Потеря информации на основе энтропии 0.0
My by element distance: 0.0
Utility boosting score: 0.8339279440136041


0.8339279440136041

### Для сгенерированного

In [197]:
score_depersonalization_quality(df_generated,df_generated,train_generated,test_generated,train_generated,test_generated, quasi_identifiers_columns_ids_generated,quasi_identifiers_columns_types_generated,sensetive_columns_ids_generated,sensetive_columns_types_generated,predicted_column_generated,quasi_identifiers_boosting_ids_generated, sensitive_boosting_ids_generated,generalization=False, short = False, classification=False)[6]

Максимальное k: 1
Максимальное l: 1
Минимальное t: 0.5040897912983124
Adversarial knowledge gain: 0.9989939999999765
Adversarial accuracy gain: 0.9980000000000008
Средний размер класса эквивалентности: 1.0
Distinctness: 1.0
Доля изменённых значений: 0.0
Неоднородная энтропия: 9.965784284662083
Потеря информации на основе энтропии 0.0
My by element distance: 0.0
Utility boosting score: 1.6312812451979934


1.6312812451979934

## Алгоритм time optimal

### Подавление

#### K-анонимность

In [218]:
scores_dfs = []
for k, k_short in zip(k_s,k_s_short):
    print(f"Обезличивание сгенерированного датасета c k = {k}\n")
    dep = SuppressionKAnonymityTimeOptimal(k)
    scores_generated = run_test_on_depersonalizer(np.copy(df_generated), np.copy(train_generated), np.copy(test_generated), dep, quasi_identifiers_columns_ids_generated, quasi_identifiers_columns_types_generated,sensetive_columns_ids_generated,sensetive_columns_types_generated, predicted_column_generated,quasi_identifiers_boosting_ids_generated, sensitive_boosting_ids_generated,classification=classification_generated)
    print(f"\nОбезличивание датасета с caggle c k = {k}\n")
    scores_caggle = run_test_on_depersonalizer(np.copy(df_caggle), np.copy(train_caggle), np.copy(test_caggle), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle, quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle,classification=classification_caggle)
    print(f"\nОбезличивание короткого датасета c k = {k_short}\n")
    dep = SuppressionKAnonymityTimeOptimal(k_short)
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(pd.DataFrame([scores_generated, scores_caggle, scores_short], index, columns))

Обезличивание сгенерированного датасета c k = 2

Время выполнения: 1.9748902320861816 секунд
Максимальное k: 2
Максимальное l: 1
Минимальное t: 0.480619714218434
Adversarial knowledge gain: 0.9979879999999776
Adversarial accuracy gain: 0.4980000000000004
Средний размер класса эквивалентности: 2.0
Distinctness: 0.5
Доля изменённых значений: 0.5294666666666666
Неоднородная энтропия: 8.965784284662082
Потеря информации на основе энтропии 0.10034333188799394
My by element distance: 0.5294666666666666
Utility boosting score: 1.2059329558863494

Обезличивание датасета с caggle c k = 2

Время выполнения: 1.7030000686645508 секунд
Максимальное k: 2
Максимальное l: 1
Минимальное t: 0.6146789999938533
Adversarial knowledge gain: 0.9692499999999965
Adversarial accuracy gain: 0.4660000000000004
Средний размер класса эквивалентности: 2.288329519450801
Distinctness: 0.437
Доля изменённых значений: 0.3564
Неоднородная энтропия: 8.624493878833894
Потеря информации на основе энтропии 0.1337202944301616

Показатели для k=2, k_short=2

In [220]:
scores_dfs[0]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,2.0,0.5,0.557867,8.965784,0.100343,0.174841,1.397296,2,1,0.492499,0.997988,0.498000,2.128999
caggle,2.0,0.5,0.499600,8.965784,0.099440,0.016704,0.295000,2,1,0.575579,0.980014,0.495000,1.739000
short,2.0,0.5,0.916667,2.584963,0.278943,0.090278,NaN,2,2,0.406824,0.833333,0.416667,0.001000


Показатели для k=10, k_short=3

In [221]:
scores_dfs[1]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,10.0,0.100000,0.964667,6.643856,0.333333,0.633813,1.870488,10,3,0.235635,0.98994,0.168,1.706008
caggle,10.0,0.100000,0.722000,6.643856,0.332664,0.189151,0.245000,10,1,0.260893,0.90409,0.173,1.608000
short,3.0,0.333333,1.000000,2.000000,0.442114,0.088963,NaN,3,3,0.263780,0.75000,0.250,0.000999


Показатели для k=100, k_short=4

In [222]:
scores_dfs[2]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,100.0,0.01,1.0,3.321928,0.666667,0.816057,1.724434,100,4,0.039672,0.899400,0.035000,1.647999
caggle,100.0,0.01,1.0,3.321928,0.666332,0.750283,0.255000,100,24,0.118355,0.455600,0.026000,1.568008
short,4.0,0.25,1.0,1.584963,0.557886,0.132639,NaN,4,4,0.259186,0.666667,0.166667,0.000000


#### L-разнообразие

In [223]:
scores_dfs = []
for k, k_short, l in zip(k_s,k_s_short,l_s):
    print(f"Обезличивание сгенерированного датасета c k = {k}, l = {l}\n")
    dep = SuppressionLDiversityTimeOptimal(k,l)
    scores_generated = run_test_on_depersonalizer(np.copy(df_generated), np.copy(train_generated), np.copy(test_generated), dep, quasi_identifiers_columns_ids_generated, quasi_identifiers_columns_types_generated,sensetive_columns_ids_generated,sensetive_columns_types_generated, predicted_column_generated,quasi_identifiers_boosting_ids_generated, sensitive_boosting_ids_generated,classification=classification_generated)
    print(f"\nОбезличивание датасета с caggle c k = {k}, l = {l}\n")
    scores_caggle = run_test_on_depersonalizer(np.copy(df_caggle), np.copy(train_caggle), np.copy(test_caggle), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle, quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle,classification=classification_caggle)
    print(f"\nОбезличивание короткого датасета c k = {k_short}, l = {l}\n")
    dep = SuppressionLDiversityTimeOptimal(k_short, l)
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(pd.DataFrame([scores_generated, scores_caggle, scores_short], index, columns))

Обезличивание сгенерированного датасета c k = 2, l = 2

Время выполнения: 1.758016586303711 секунд
Максимальное k: 2
Максимальное l: 2
Минимальное t: 0.4897177644330452
Adversarial knowledge gain: 0.9974719999999756
Adversarial accuracy gain: 0.4310000000000003
Средний размер класса эквивалентности: 2.3094688221709005
Distinctness: 0.433
Доля изменённых значений: 0.5863333333333334
Неоднородная энтропия: 8.70740310360697
Потеря информации на основе энтропии 0.12627016049221884
My by element distance: 0.5863333333333334
Utility boosting score: 2.1460130514172295

Обезличивание датасета с caggle c k = 2, l = 2

Время выполнения: 1.6329996585845947 секунд
Максимальное k: 2
Максимальное l: 2
Минимальное t: 0.6021329011344244
Adversarial knowledge gain: 0.7281379999999977
Adversarial accuracy gain: 0.19600000000000015
Средний размер класса эквивалентности: 5.9523809523809526
Distinctness: 0.168
Доля изменённых значений: 0.6566
Неоднородная энтропия: 5.837208105163705
Потеря информации на ос

Показатели для k=2, k_short=2, l=2

In [224]:
scores_dfs[0]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,2.309469,0.433000,0.586333,8.707403,0.126270,0.586333,2.146013,2,2,0.489718,0.997472,0.431000,1.758017
caggle,5.952381,0.168000,0.656600,5.837208,0.413687,0.656600,0.255000,2,2,0.602133,0.728138,0.196000,1.633000
short,6.000000,0.166667,0.916667,0.650022,0.818681,0.916667,NaN,2,2,0.292110,0.277778,0.083333,0.000000


Показатели для k=10, k_short=3, l=3

In [225]:
scores_dfs[1]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,45.454545,0.022000,0.950667,2.617839,0.737317,0.950667,2.861081,10,3,0.175965,0.637980,0.050,1.671000
caggle,58.823529,0.017000,0.859000,2.989632,0.699709,0.859000,0.245000,10,3,0.280183,0.413267,0.034,1.587998
short,12.000000,0.083333,1.000000,0.000000,1.000000,1.000000,NaN,12,9,0.000000,0.000000,0.000,0.000000


Показатели для k=100, k_short=4, l=4

In [226]:
scores_dfs[2]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,1000.0,0.001000,1.0,0.0,1.0,1.0,1.857825,1000,4,0,0.0,0.0,1.633006
caggle,1000.0,0.001000,1.0,0.0,1.0,1.0,0.245000,1000,90,0,0.0,0.0,1.551037
short,12.0,0.083333,1.0,0.0,1.0,1.0,NaN,12,9,0,0.0,0.0,0.000000


### Обобщение

#### K-анонимность

In [227]:
scores_dfs = []
for k, k_short in zip(k_s,k_s_short):
    print(f"Обезличивание сгенерированного датасета c k = {k}\n")
    dep = GeneralizationKAnonymityTimeOptimal(k,quasi_identifiers_columns_types_generated)
    scores_generated = run_test_on_depersonalizer(np.copy(df_generated), np.copy(train_generated), np.copy(test_generated), dep, quasi_identifiers_columns_ids_generated, quasi_identifiers_columns_types_generated,sensetive_columns_ids_generated,sensetive_columns_types_generated, predicted_column_generated,quasi_identifiers_boosting_ids_generated, sensitive_boosting_ids_generated,classification=classification_generated,generalization=True)
    print(f"\nОбезличивание датасета с caggle c k = {k}\n")
    dep = GeneralizationKAnonymityTimeOptimal(k,quasi_identifiers_columns_types_caggle)
    scores_caggle = run_test_on_depersonalizer(np.copy(df_caggle), np.copy(train_caggle), np.copy(test_caggle), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle, quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle,classification=classification_caggle,generalization=True)
    print(f"\nОбезличивание короткого датасета c k = {k_short}\n")
    dep = GeneralizationKAnonymityTimeOptimal(k_short,quasi_identifiers_columns_types_caggle[:2])
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True,generalization=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(pd.DataFrame([scores_generated, scores_caggle, scores_short], index, columns))

Обезличивание сгенерированного датасета c k = 2

Время выполнения: 1.7996547222137451 секунд
Максимальное k: 2
Максимальное l: 1
Минимальное t: 0.49249899079947246
Adversarial knowledge gain: 0.9979879999999771
Adversarial accuracy gain: 0.4980000000000004
Средний размер класса эквивалентности: 2.0
Distinctness: 0.5
Доля изменённых значений: 0.5578666666666666
Неоднородная энтропия: 8.965784284662082
Потеря информации на основе энтропии 0.10034333188799394
My by element distance: 0.1748410090535244
Utility boosting score: 1.397295607295797

Обезличивание датасета с caggle c k = 2

Время выполнения: 1.7319998741149902 секунд
Максимальное k: 2
Максимальное l: 1
Минимальное t: 0.5755789999942441
Adversarial knowledge gain: 0.980013999999998
Adversarial accuracy gain: 0.49500000000000044
Средний размер класса эквивалентности: 2.0
Distinctness: 0.5
Доля изменённых значений: 0.4996
Неоднородная энтропия: 8.965784284662082
Потеря информации на основе энтропии 0.09943967965690048
My by element

Показатели для k=2, k_short=2

In [228]:
scores_dfs[0]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,2.0,0.5,0.557867,8.965784,0.100343,0.174841,1.397296,2,1,0.492499,0.997988,0.498000,1.799655
caggle,2.0,0.5,0.499600,8.965784,0.099440,0.016704,0.295000,2,1,0.575579,0.980014,0.495000,1.732000
short,2.0,0.5,0.916667,2.584963,0.278943,0.090278,NaN,2,2,0.406824,0.833333,0.416667,0.000000


Показатели для k=10, k_short=3

In [229]:
scores_dfs[1]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,10.0,0.100000,0.964667,6.643856,0.333333,0.633813,1.870488,10,3,0.235635,0.98994,0.168,1.667721
caggle,10.0,0.100000,0.722000,6.643856,0.332664,0.189151,0.245000,10,1,0.260893,0.90409,0.173,1.611000
short,3.0,0.333333,1.000000,2.000000,0.442114,0.088963,NaN,3,3,0.263780,0.75000,0.250,0.000000


Показатели для k=100, k_short=4

In [230]:
scores_dfs[2]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,100.0,0.01,1.0,3.321928,0.666667,0.816057,1.724434,100,4,0.039672,0.899400,0.035000,1.634012
caggle,100.0,0.01,1.0,3.321928,0.666332,0.750283,0.255000,100,24,0.118355,0.455600,0.026000,1.573431
short,4.0,0.25,1.0,1.584963,0.557886,0.132639,NaN,4,4,0.259186,0.666667,0.166667,0.000000


#### L-разнообразие

In [231]:
scores_dfs = []
for k, k_short,l in zip(k_s,k_s_short,l_s):
    print(f"Обезличивание сгенерированного датасета c k = {k}, l = {l}\n")
    dep = GeneralizationLDiversityTimeOptimal(k,l,quasi_identifiers_columns_types_generated)
    scores_generated = run_test_on_depersonalizer(np.copy(df_generated), np.copy(train_generated), np.copy(test_generated), dep, quasi_identifiers_columns_ids_generated, quasi_identifiers_columns_types_generated,sensetive_columns_ids_generated,sensetive_columns_types_generated, predicted_column_generated,quasi_identifiers_boosting_ids_generated, sensitive_boosting_ids_generated,classification=classification_generated,generalization=True)
    print(f"\nОбезличивание датасета с caggle c k = {k}, l = {l}\n")
    dep = GeneralizationLDiversityTimeOptimal(k,l,quasi_identifiers_columns_types_caggle)
    scores_caggle = run_test_on_depersonalizer(np.copy(df_caggle), np.copy(train_caggle), np.copy(test_caggle), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle, quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle,classification=classification_caggle,generalization=True)
    print(f"\nОбезличивание короткого датасета c k = {k_short}, l = {l}\n")
    dep = GeneralizationLDiversityTimeOptimal(k_short,l,quasi_identifiers_columns_types_caggle[:2])
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True,generalization=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(pd.DataFrame([scores_generated, scores_caggle, scores_short], index, columns))

Обезличивание сгенерированного датасета c k = 2, l = 2

Время выполнения: 1.7814218997955322 секунд
Максимальное k: 2
Максимальное l: 2
Минимальное t: 0.46067573861955086
Adversarial knowledge gain: 0.9974129999999752
Adversarial accuracy gain: 0.4190000000000003
Средний размер класса эквивалентности: 2.375296912114014
Distinctness: 0.421
Доля изменённых значений: 0.6340666666666667
Неоднородная энтропия: 8.666019025621715
Потеря информации на основе энтропии 0.13042277676437186
My by element distance: 0.23638380371793205
Utility boosting score: 2.1231831380197663

Обезличивание датасета с caggle c k = 2, l = 2

Время выполнения: 1.6619977951049805 секунд
Максимальное k: 2
Максимальное l: 2
Минимальное t: 0.5755789999942441
Adversarial knowledge gain: 0.9505369999999977
Adversarial accuracy gain: 0.3180000000000003
Средний размер класса эквивалентности: 3.676470588235294
Distinctness: 0.272
Доля изменённых значений: 0.6242
Неоднородная энтропия: 7.851702027429677
Потеря информации на о

Показатели для k=2, k_short=2, l=2

In [232]:
scores_dfs[0]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,2.375297,0.421,0.634067,8.666019,0.130423,0.236384,2.123183,2,2,0.460676,0.997413,0.419000,1.781422
caggle,3.676471,0.272,0.624200,7.851702,0.211343,0.087389,0.275000,2,2,0.575579,0.950537,0.318000,1.661998
short,2.000000,0.500,0.916667,2.584963,0.278943,0.090278,NaN,2,2,0.406824,0.833333,0.416667,0.000999


Показатели для k=10, k_short=3, l=3

In [233]:
scores_dfs[1]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,10.000000,0.100000,0.964667,6.643856,0.333333,0.633813,1.966712,10,3,0.235635,0.989940,0.168,1.683000
caggle,10.526316,0.095000,0.730800,6.558559,0.341231,0.212042,0.190000,10,3,0.260893,0.897345,0.166,1.605998
short,3.000000,0.333333,1.000000,2.000000,0.442114,0.088963,NaN,3,3,0.263780,0.750000,0.250,0.001000


Показатели для k=100, k_short=4, l=4

In [234]:
scores_dfs[2]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,100.0,0.01,1.0,3.321928,0.666667,0.816057,1.724434,100,4,0.039672,0.899400,0.035000,1.617153
caggle,100.0,0.01,1.0,3.321928,0.666332,0.750283,0.255000,100,24,0.118355,0.455600,0.026000,1.588008
short,4.0,0.25,1.0,1.584963,0.557886,0.132639,NaN,4,4,0.259186,0.666667,0.166667,0.001002


### Агрегация

#### K-анонимность

In [235]:
scores_dfs = []
for k, k_short in zip(k_s,k_s_short):
    print(f"Обезличивание сгенерированного датасета c k = {k}\n")
    dep = AggregationKAnonymityTimeOptimal(k,quasi_identifiers_columns_types_generated)
    scores_generated = run_test_on_depersonalizer(np.copy(df_generated), np.copy(train_generated), np.copy(test_generated), dep, quasi_identifiers_columns_ids_generated, quasi_identifiers_columns_types_generated,sensetive_columns_ids_generated,sensetive_columns_types_generated, predicted_column_generated,quasi_identifiers_boosting_ids_generated, sensitive_boosting_ids_generated,classification=classification_generated)
    print(f"\nОбезличивание датасета с caggle c k = {k}\n")
    dep = AggregationKAnonymityTimeOptimal(k,quasi_identifiers_columns_types_caggle)
    scores_caggle = run_test_on_depersonalizer(np.copy(df_caggle), np.copy(train_caggle), np.copy(test_caggle), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle, quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle,classification=classification_caggle)
    print(f"\nОбезличивание короткого датасета c k = {k_short}\n")
    dep = AggregationKAnonymityTimeOptimal(k_short,quasi_identifiers_columns_types_caggle[:2])
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(pd.DataFrame([scores_generated, scores_caggle, scores_short], index, columns))

Обезличивание сгенерированного датасета c k = 2

Время выполнения: 1.8209481239318848 секунд
Максимальное k: 2
Максимальное l: 1
Минимальное t: 0.49249899079947246
Adversarial knowledge gain: 0.9979879999999771
Adversarial accuracy gain: 0.4980000000000004
Средний размер класса эквивалентности: 2.0
Distinctness: 0.5
Доля изменённых значений: 0.3737333333333333
Неоднородная энтропия: 8.965784284662082
Потеря информации на основе энтропии 0.10034333188799394
My by element distance: 0.1748410090535244
Utility boosting score: 2.2856730066496893

Обезличивание датасета с caggle c k = 2

Время выполнения: 1.7219998836517334 секунд
Максимальное k: 2
Максимальное l: 1
Минимальное t: 0.5755789999942441
Adversarial knowledge gain: 0.980013999999998
Adversarial accuracy gain: 0.49500000000000044
Средний размер класса эквивалентности: 2.0
Distinctness: 0.5
Доля изменённых значений: 0.4036
Неоднородная энтропия: 8.965784284662082
Потеря информации на основе энтропии 0.09943967965690048
My by elemen

Показатели для k=2, k_short=2

In [236]:
scores_dfs[0]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,2.0,0.5,0.373733,8.965784,0.100343,0.174841,2.285673,2,1,0.492499,0.997988,0.498000,1.820948
caggle,2.0,0.5,0.403600,8.965784,0.099440,0.016704,0.245000,2,1,0.575579,0.980014,0.495000,1.722000
short,2.0,0.5,0.916667,2.584963,0.278943,0.090278,NaN,2,2,0.406824,0.833333,0.416667,0.000000


Показатели для k=10, k_short=3

In [237]:
scores_dfs[1]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,10.0,0.100000,0.633400,6.643856,0.333333,0.364309,1.915734,10,3,0.235635,0.98994,0.168,1.666000
caggle,10.0,0.100000,0.685000,6.643856,0.332664,0.061644,0.275000,10,1,0.260893,0.90409,0.173,1.603002
short,3.0,0.333333,0.958333,2.000000,0.442114,0.070370,NaN,3,3,0.263780,0.75000,0.250,0.000000


Показатели для k=100, k_short=4

In [238]:
scores_dfs[2]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,100.0,0.01,0.760933,3.321928,0.666667,0.492410,1.927439,100,4,0.039672,0.899400,0.035000,1.625999
caggle,100.0,0.01,0.996200,3.321928,0.666332,0.172376,0.250000,100,24,0.118355,0.455600,0.026000,1.577000
short,4.0,0.25,1.000000,1.584963,0.557886,0.104861,NaN,4,4,0.259186,0.666667,0.166667,0.000000


#### L-разнообразие

In [239]:
scores_dfs = []
for k, k_short,l in zip(k_s,k_s_short,l_s):
    print(f"Обезличивание сгенерированного датасета c k = {k}, l = {l}\n")
    dep = AggregationLDiversityTimeOptimal(k,l,quasi_identifiers_columns_types_generated)
    scores_generated = run_test_on_depersonalizer(np.copy(df_generated), np.copy(train_generated), np.copy(test_generated), dep, quasi_identifiers_columns_ids_generated, quasi_identifiers_columns_types_generated,sensetive_columns_ids_generated,sensetive_columns_types_generated, predicted_column_generated,quasi_identifiers_boosting_ids_generated, sensitive_boosting_ids_generated,classification=classification_generated)
    print(f"\nОбезличивание датасета с caggle c k = {k}, l = {l}\n")
    dep = AggregationLDiversityTimeOptimal(k,l,quasi_identifiers_columns_types_caggle)
    scores_caggle = run_test_on_depersonalizer(np.copy(df_caggle), np.copy(train_caggle), np.copy(test_caggle), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle, quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle,classification=classification_caggle)
    print(f"\nОбезличивание короткого датасета c k = {k_short}, l = {l}\n")
    dep = AggregationLDiversityTimeOptimal(k_short,l,quasi_identifiers_columns_types_caggle[:2])
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(pd.DataFrame([scores_generated, scores_caggle, scores_short], index, columns))

Обезличивание сгенерированного датасета c k = 2, l = 2

Время выполнения: 1.7730305194854736 секунд
Максимальное k: 2
Максимальное l: 2
Минимальное t: 0.46067573861955086
Adversarial knowledge gain: 0.9974129999999752
Adversarial accuracy gain: 0.4190000000000003
Средний размер класса эквивалентности: 2.375296912114014
Distinctness: 0.421
Доля изменённых значений: 0.416
Неоднородная энтропия: 8.666019025621715
Потеря информации на основе энтропии 0.13042277676437186
My by element distance: 0.20377638952478103
Utility boosting score: 1.8903232471828957

Обезличивание датасета с caggle c k = 2, l = 2

Время выполнения: 1.6499977111816406 секунд
Максимальное k: 2
Максимальное l: 2
Минимальное t: 0.5755789999942441
Adversarial knowledge gain: 0.9505369999999977
Adversarial accuracy gain: 0.3180000000000003
Средний размер класса эквивалентности: 3.676470588235294
Distinctness: 0.272
Доля изменённых значений: 0.55
Неоднородная энтропия: 7.851702027429676
Потеря информации на основе энтропии 

Показатели для k=2, k_short=2, l=2

In [240]:
scores_dfs[0]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,2.375297,0.421,0.416000,8.666019,0.130423,0.203776,1.890323,2,2,0.460676,0.997413,0.419000,1.773031
caggle,3.676471,0.272,0.550000,7.851702,0.211343,0.036458,0.235000,2,2,0.575579,0.950537,0.318000,1.649998
short,2.000000,0.500,0.916667,2.584963,0.278943,0.090278,NaN,2,2,0.406824,0.833333,0.416667,0.000000


Показатели для k=10, k_short=3, l=3

In [241]:
scores_dfs[1]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,10.000000,0.100000,0.633400,6.643856,0.333333,0.364309,1.885908,10,3,0.235635,0.989940,0.168,1.679
caggle,10.526316,0.095000,0.693400,6.558559,0.341231,0.064193,0.250000,10,3,0.260893,0.897345,0.166,1.596
short,3.000000,0.333333,0.958333,2.000000,0.442114,0.070370,NaN,3,3,0.263780,0.750000,0.250,0.000


Показатели для k=100, k_short=4, l=4

In [242]:
scores_dfs[2]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,100.0,0.01,0.760933,3.321928,0.666667,0.492410,1.927439,100,4,0.039672,0.899400,0.035000,1.646999
caggle,100.0,0.01,0.996200,3.321928,0.666332,0.172376,0.250000,100,24,0.118355,0.455600,0.026000,1.566000
short,4.0,0.25,1.000000,1.584963,0.557886,0.104861,NaN,4,4,0.259186,0.666667,0.166667,0.001000


## Алгоритм group join

### Подавление

#### K-анонимность

In [243]:
scores_dfs = []
for k, k_short in zip(k_s,k_s_short):
    print(f"Обезличивание сгенерированного датасета c k = {k}\n")
    method = GroupJoinSuppression()
    metric = GroupJoinKAnonymity(k, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_generated = run_test_on_depersonalizer(np.copy(df_generated), np.copy(train_generated), np.copy(test_generated), dep, quasi_identifiers_columns_ids_generated, quasi_identifiers_columns_types_generated,sensetive_columns_ids_generated,sensetive_columns_types_generated, predicted_column_generated,quasi_identifiers_boosting_ids_generated, sensitive_boosting_ids_generated,classification=classification_generated)
    print(f"\nОбезличивание датасета с caggle c k = {k}\n")
    scores_caggle = run_test_on_depersonalizer(np.copy(df_caggle), np.copy(train_caggle), np.copy(test_caggle), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle, quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle,classification=classification_caggle)
    print(f"\nОбезличивание короткого датасета c k = {k_short}\n")
    metric = GroupJoinKAnonymity(k_short, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(pd.DataFrame([scores_generated, scores_caggle, scores_short], index, columns))

Обезличивание сгенерированного датасета c k = 2

Время выполнения: 4.989999055862427 секунд
Максимальное k: 2
Максимальное l: 1
Минимальное t: 0.4376954468278831
Adversarial knowledge gain: 0.9874489999999744
Adversarial accuracy gain: 0.1930000000000003
Средний размер класса эквивалентности: 6.024096385542169
Distinctness: 0.166
Доля изменённых значений: 0.6878
Неоднородная энтропия: 6.907872597457532
Потеря информации на основе энтропии 0.3068410473133413
My by element distance: 0.6878
Utility boosting score: 2.07817046897518

Обезличивание датасета с caggle c k = 2

Время выполнения: 2.67100191116333 секунд
Максимальное k: 2
Максимальное l: 1
Минимальное t: 0.6339492385783585
Adversarial knowledge gain: 0.9428889999999965
Adversarial accuracy gain: 0.41500000000000037
Средний размер класса эквивалентности: 2.9154518950437316
Distinctness: 0.343
Доля изменённых значений: 0.359
Неоднородная энтропия: 8.057446441424611
Потеря информации на основе энтропии 0.1906768757698033
My by eleme

Показатели для k=2, k_short=2

In [244]:
scores_dfs[0]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,6.024096,0.166000,0.687800,6.907873,0.306841,0.687800,2.07817,2,1,0.437695,0.987449,0.193000,4.989999
caggle,2.915452,0.343000,0.359000,8.057446,0.190677,0.359000,0.24500,2,1,0.633949,0.942889,0.415000,2.671002
short,6.000000,0.166667,0.916667,0.650022,0.818681,0.916667,NaN,2,2,0.345144,0.277778,0.083333,0.001000


Показатели для k=10, k_short=3

In [245]:
scores_dfs[1]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,16.666667,0.060000,0.780933,5.804813,0.417526,0.780933,1.564202,10,3,0.255205,0.980532,0.132,5.128001
caggle,33.333333,0.030000,0.688200,4.630698,0.534874,0.688200,0.255000,10,3,0.241827,0.667726,0.066,3.198000
short,12.000000,0.083333,1.000000,0.000000,1.000000,1.000000,NaN,12,9,0.000000,0.000000,0.000,0.001000


Показатели для k=100, k_short=4

In [246]:
scores_dfs[2]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,200.000000,0.005000,0.9362,2.109269,0.788349,0.9362,1.857825,122,4,0.024405,0.728102,0.042,5.294008
caggle,166.666667,0.006000,0.8426,2.519868,0.746894,0.8426,0.260000,108,28,0.058945,0.328849,0.017,3.153000
short,12.000000,0.083333,1.0000,0.000000,1.000000,1.0000,NaN,12,9,0.000000,0.000000,0.000,0.000999


#### L-разнообразие

In [247]:
scores_dfs = []
for k, k_short, l in zip(k_s,k_s_short,l_s):
    print(f"Обезличивание сгенерированного датасета c k = {k}, l = {l}\n")
    method = GroupJoinSuppression()
    metric = GroupJoinLDiversity(k, l, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_generated = run_test_on_depersonalizer(np.copy(df_generated), np.copy(train_generated), np.copy(test_generated), dep, quasi_identifiers_columns_ids_generated, quasi_identifiers_columns_types_generated,sensetive_columns_ids_generated,sensetive_columns_types_generated, predicted_column_generated,quasi_identifiers_boosting_ids_generated, sensitive_boosting_ids_generated,classification=classification_generated)
    print(f"\nОбезличивание датасета с caggle c k = {k}, l = {l}\n")
    scores_caggle = run_test_on_depersonalizer(np.copy(df_caggle), np.copy(train_caggle), np.copy(test_caggle), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle, quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle,classification=classification_caggle)
    print(f"\nОбезличивание короткого датасета c k = {k_short}, l = {l}\n")
    metric = GroupJoinLDiversity(k_short, l, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(pd.DataFrame([scores_generated, scores_caggle, scores_short], index, columns))

Обезличивание сгенерированного датасета c k = 2, l = 2

Время выполнения: 7.375264644622803 секунд
Максимальное k: 2
Максимальное l: 2
Минимальное t: 0.46413225284602233
Adversarial knowledge gain: 0.9900269999999747
Adversarial accuracy gain: 0.19000000000000028
Средний размер класса эквивалентности: 6.25
Distinctness: 0.16
Доля изменённых значений: 0.6912
Неоднородная энтропия: 6.97467461387654
Потеря информации на основе энтропии 0.3001379104090216
My by element distance: 0.6912000000000001
Utility boosting score: 2.555978731472018

Обезличивание датасета с caggle c k = 2, l = 2

Время выполнения: 5.434001445770264 секунд
Максимальное k: 2
Максимальное l: 2
Минимальное t: 0.5755789999942441
Adversarial knowledge gain: 0.8848109999999973
Adversarial accuracy gain: 0.26300000000000023
Средний размер класса эквивалентности: 5.025125628140704
Distinctness: 0.199
Доля изменённых значений: 0.4792
Неоднородная энтропия: 7.0228354853014645
Потеря информации на основе энтропии 0.294597463695

Показатели для k=2, k_short=2, l=2

In [248]:
scores_dfs[0]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,6.250000,0.160000,0.691200,6.974675,0.300138,0.691200,2.555979,2,2,0.464132,0.990027,0.190000,7.375265
caggle,5.025126,0.199000,0.479200,7.022835,0.294597,0.479200,0.245000,2,2,0.575579,0.884811,0.263000,5.434001
short,6.000000,0.166667,0.916667,0.650022,0.818681,0.916667,NaN,2,2,0.345144,0.277778,0.083333,0.001000


Показатели для k=10, k_short=3, l=3

In [249]:
scores_dfs[1]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,250.000000,0.004000,0.987733,0.378212,0.962049,0.987733,1.847724,10,4,0.113898,0.102142,0.003,7.249572
caggle,43.478261,0.023000,0.678000,4.143000,0.583860,0.678000,0.245000,10,3,0.150551,0.588525,0.062,5.696785
short,12.000000,0.083333,1.000000,0.000000,1.000000,1.000000,NaN,12,9,0.000000,0.000000,0.000,0.001000


Показатели для k=100, k_short=4, l=4

In [250]:
scores_dfs[2]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,1000.000000,0.001000,1.0000,0.000000,1.000000,1.0000,1.857825,1000,4,0.000000,0.000000,0.000,7.233000
caggle,166.666667,0.006000,0.8188,2.528811,0.745996,0.8188,0.260000,102,22,0.082743,0.353515,0.015,5.826014
short,12.000000,0.083333,1.0000,0.000000,1.000000,1.0000,NaN,12,9,0.000000,0.000000,0.000,0.000999


#### T-близость

In [251]:
scores_dfs = []
for k, k_short, t in zip(k_s,k_s_short,t_s):
    print(f"Обезличивание сгенерированного датасета c k = {k}, t = {t}\n")
    method = GroupJoinSuppression()
    metric = GroupJoinTCloseness(k, t, sensetive_columns_types_generated, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_generated = run_test_on_depersonalizer(np.copy(df_generated), np.copy(train_generated), np.copy(test_generated), dep, quasi_identifiers_columns_ids_generated, quasi_identifiers_columns_types_generated,sensetive_columns_ids_generated,sensetive_columns_types_generated, predicted_column_generated,quasi_identifiers_boosting_ids_generated, sensitive_boosting_ids_generated,classification=classification_generated)
    print(f"\nОбезличивание датасета с caggle c k = {k}, t = {t}\n")
    metric = GroupJoinTCloseness(k, t, sensetive_columns_types_caggle, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_caggle = run_test_on_depersonalizer(np.copy(df_caggle), np.copy(train_caggle), np.copy(test_caggle), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle, quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle,classification=classification_caggle)
    print(f"\nОбезличивание короткого датасета c k = {k_short}, t = {t}\n")
    metric = GroupJoinTCloseness(k_short, t, sensetive_columns_types_caggle, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(pd.DataFrame([scores_generated, scores_caggle, scores_short], index, columns))

Обезличивание сгенерированного датасета c k = 2, t = 1

Время выполнения: 266.5385241508484 секунд
Максимальное k: 4
Максимальное l: 4
Минимальное t: 0.08711735273936147
Adversarial knowledge gain: 0.007967999999999727
Adversarial accuracy gain: 0.001
Средний размер класса эквивалентности: 500.0
Distinctness: 0.002
Доля изменённых значений: 0.9997333333333334
Неоднородная энтропия: 0.037622360322278434
Потеря информации на основе энтропии 0.9962248470117719
My by element distance: 0.9997333333333334
Utility boosting score: 1.8578250732839592

Обезличивание датасета с caggle c k = 2, t = 1

Время выполнения: 153.24030828475952 секунд
Максимальное k: 2
Максимальное l: 1
Минимальное t: 0.3552285251214983
Adversarial knowledge gain: 0.27992199999999967
Adversarial accuracy gain: 0.07000000000000005
Средний размер класса эквивалентности: 17.24137931034483
Distinctness: 0.058
Доля изменённых значений: 0.857
Неоднородная энтропия: 2.221303021952093
Потеря информации на основе энтропии 0.77688

Показатели для k=2, k_short=2, t=1

In [252]:
scores_dfs[0]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,500.000000,0.002000,0.999733,0.037622,0.996225,0.999733,1.857825,4,4,0.087117,0.007968,0.001,266.538524
caggle,17.241379,0.058000,0.857000,2.221303,0.776883,0.857000,0.245000,2,1,0.355229,0.279922,0.070,153.240308
short,12.000000,0.083333,1.000000,0.000000,1.000000,1.000000,NaN,12,9,0.000000,0.000000,0.000,0.004999


Показатели для k=10, k_short=3, t=0.5

In [253]:
scores_dfs[1]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,1000.0,0.001000,1.0,0.0,1.0,1.0,1.857825,1000,4,0,0.0,0.0,266.164511
caggle,1000.0,0.001000,1.0,0.0,1.0,1.0,0.245000,1000,90,0,0.0,0.0,153.466587
short,12.0,0.083333,1.0,0.0,1.0,1.0,NaN,12,9,0,0.0,0.0,0.004000


Показатели для k=100, k_short=4, t=0.1

In [254]:
scores_dfs[2]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,1000.0,0.001000,1.0,0.0,1.0,1.0,1.857825,1000,4,0,0.0,0.0,265.434979
caggle,1000.0,0.001000,1.0,0.0,1.0,1.0,0.245000,1000,90,0,0.0,0.0,154.019700
short,12.0,0.083333,1.0,0.0,1.0,1.0,NaN,12,9,0,0.0,0.0,0.005000


### Обобщение

#### K-анонимность

In [255]:
scores_dfs = []
for k, k_short in zip(k_s,k_s_short):
    print(f"Обезличивание сгенерированного датасета c k = {k}\n")
    method = GroupJoinGeneralization(quasi_identifiers_columns_types_generated)
    metric = GroupJoinKAnonymity(k, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_generated = run_test_on_depersonalizer(np.copy(df_generated), np.copy(train_generated), np.copy(test_generated), dep, quasi_identifiers_columns_ids_generated, quasi_identifiers_columns_types_generated,sensetive_columns_ids_generated,sensetive_columns_types_generated, predicted_column_generated,quasi_identifiers_boosting_ids_generated, sensitive_boosting_ids_generated,classification=classification_generated,generalization=True)
    print(f"\nОбезличивание датасета с caggle c k = {k}\n")
    method = GroupJoinGeneralization(quasi_identifiers_columns_types_caggle)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_caggle = run_test_on_depersonalizer(np.copy(df_caggle), np.copy(train_caggle), np.copy(test_caggle), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle, quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle,classification=classification_caggle,generalization=True)
    print(f"\nОбезличивание короткого датасета c k = {k_short}\n")
    method = GroupJoinGeneralization(quasi_identifiers_columns_types_caggle[:2])
    metric = GroupJoinKAnonymity(k_short, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True,generalization=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(pd.DataFrame([scores_generated, scores_caggle, scores_short], index, columns))

Обезличивание сгенерированного датасета c k = 2

Время выполнения: 10.822349309921265 секунд
Максимальное k: 2
Максимальное l: 1
Минимальное t: 0.4819493446457039
Adversarial knowledge gain: 0.9978119999999776
Adversarial accuracy gain: 0.46900000000000036
Средний размер класса эквивалентности: 2.1231422505307855
Distinctness: 0.471
Доля изменённых значений: 0.5702
Неоднородная энтропия: 8.86351058454093
Потеря информации на основе энтропии 0.11060581572266392
My by element distance: 0.18967777265927682
Utility boosting score: 1.49929314471002

Обезличивание датасета с caggle c k = 2

Время выполнения: 9.813027381896973 секунд
Максимальное k: 2
Максимальное l: 1
Минимальное t: 0.5822456666608443
Adversarial knowledge gain: 0.9743119999999978
Adversarial accuracy gain: 0.4410000000000004
Средний размер класса эквивалентности: 2.4154589371980677
Distinctness: 0.414
Доля изменённых значений: 0.5152
Неоднородная энтропия: 8.654264831395583
Потеря информации на основе энтропии 0.13072997727

Показатели для k=2, k_short=2

In [256]:
scores_dfs[0]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,2.123142,0.471000,0.5702,8.863511,0.110606,0.189678,1.499293,2,1,0.481949,0.997812,0.469,10.822349
caggle,2.415459,0.414000,0.5152,8.654265,0.130730,0.064864,0.190000,2,1,0.582246,0.974312,0.441,9.813027
short,3.000000,0.333333,1.0000,1.959148,0.453510,0.129365,NaN,2,2,0.263780,0.736111,0.250,0.000999


Показатели для k=10, k_short=3

In [257]:
scores_dfs[1]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,15.384615,0.065,0.883067,5.956450,0.402310,0.722977,2.283622,10,3,0.170432,0.982982,0.139000,14.727130
caggle,14.705882,0.068,0.643000,6.039347,0.393383,0.363672,0.070000,10,2,0.228883,0.858729,0.153000,12.605086
short,4.000000,0.250,1.000000,1.554585,0.566359,0.243254,NaN,3,3,0.263780,0.652778,0.166667,0.001000


Показатели для k=100, k_short=4

In [258]:
scores_dfs[2]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,166.666667,0.006,0.974067,2.461750,0.752980,0.901528,1.903384,102,4,0.048750,0.800566,0.027000,14.988508
caggle,166.666667,0.006,0.944600,2.499600,0.748930,0.688648,0.255000,107,20,0.138488,0.341639,0.018000,12.859238
short,4.000000,0.250,1.000000,1.584963,0.557886,0.169378,NaN,4,3,0.171916,0.666667,0.166667,0.001000


#### L-разнообразие

In [259]:
scores_dfs = []
for k, k_short, l in zip(k_s,k_s_short,l_s):
    print(f"Обезличивание сгенерированного датасета c k = {k}, l = {l}\n")
    method = GroupJoinGeneralization(quasi_identifiers_columns_types_generated)
    metric = GroupJoinLDiversity(k, l, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_generated = run_test_on_depersonalizer(np.copy(df_generated), np.copy(train_generated), np.copy(test_generated), dep, quasi_identifiers_columns_ids_generated, quasi_identifiers_columns_types_generated,sensetive_columns_ids_generated,sensetive_columns_types_generated, predicted_column_generated,quasi_identifiers_boosting_ids_generated, sensitive_boosting_ids_generated,classification=classification_generated,generalization=True)
    print(f"\nОбезличивание датасета с caggle c k = {k}, l = {l}\n")
    method = GroupJoinGeneralization(quasi_identifiers_columns_types_caggle)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_caggle = run_test_on_depersonalizer(np.copy(df_caggle), np.copy(train_caggle), np.copy(test_caggle), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle, quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle,classification=classification_caggle,generalization=True)
    print(f"\nОбезличивание короткого датасета c k = {k_short}, l = {l}\n")
    method = GroupJoinGeneralization(quasi_identifiers_columns_types_caggle[:2])
    metric = GroupJoinLDiversity(k_short, l, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True,generalization=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(pd.DataFrame([scores_generated, scores_caggle, scores_short], index, columns))

Обезличивание сгенерированного датасета c k = 2, l = 2

Время выполнения: 12.86301040649414 секунд
Максимальное k: 2
Максимальное l: 2
Минимальное t: 0.4830184861491371
Adversarial knowledge gain: 0.997736999999977
Adversarial accuracy gain: 0.45800000000000035
Средний размер класса эквивалентности: 2.1739130434782608
Distinctness: 0.46
Доля изменённых значений: 0.5821333333333333
Неоднородная энтропия: 8.822942159510646
Потеря информации на основе энтропии 0.11467658665965086
My by element distance: 0.1982424549860486
Utility boosting score: 2.112508404971048

Обезличивание датасета с caggle c k = 2, l = 2

Время выполнения: 14.094351291656494 секунд
Максимальное k: 2
Максимальное l: 2
Минимальное t: 0.5787123333275462
Adversarial knowledge gain: 0.9484169999999968
Adversarial accuracy gain: 0.3240000000000003
Средний размер класса эквивалентности: 4.219409282700422
Distinctness: 0.237
Доля изменённых значений: 0.5856
Неоднородная энтропия: 7.725014796351682
Потеря информации на основ

Показатели для k=2, k_short=2, l=2

In [260]:
scores_dfs[0]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,2.173913,0.460000,0.582133,8.822942,0.114677,0.198242,2.112508,2,2,0.483018,0.997737,0.458,12.863010
caggle,4.219409,0.237000,0.585600,7.725015,0.224068,0.147238,0.275000,2,2,0.578712,0.948417,0.324,14.094351
short,3.000000,0.333333,1.000000,1.959148,0.453510,0.129365,NaN,2,2,0.263780,0.736111,0.250,0.001001


Показатели для k=10, k_short=3, l=3

In [261]:
scores_dfs[1]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,15.151515,0.066,0.889533,5.981492,0.399797,0.624157,2.542958,10,3,0.212052,0.983298,0.110000,16.889999
caggle,15.873016,0.063,0.644000,5.928754,0.404491,0.296305,0.240000,10,3,0.247845,0.847990,0.151000,15.416999
short,4.000000,0.250,1.000000,1.554585,0.566359,0.243254,NaN,3,3,0.263780,0.652778,0.166667,0.001000


Показатели для k=100, k_short=4, l=4

In [262]:
scores_dfs[2]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,142.857143,0.007000,0.961133,2.777643,0.721282,0.912310,1.959545,109,4,0.041910,0.850183,0.029000,17.137048
caggle,166.666667,0.006000,0.930400,2.544459,0.744424,0.330121,0.250000,111,32,0.127113,0.352789,0.027000,15.743789
short,6.000000,0.166667,1.000000,0.918296,0.743848,0.234086,NaN,4,4,0.171916,0.444444,0.083333,0.001998


#### T-близость

In [263]:
scores_dfs = []
for k, k_short, t in zip(k_s,k_s_short,t_s):
    print(f"Обезличивание сгенерированного датасета c k = {k}, t = {t}\n")
    method = GroupJoinGeneralization(quasi_identifiers_columns_types_generated)
    metric = GroupJoinTCloseness(k, t, sensetive_columns_types_generated, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_generated = run_test_on_depersonalizer(np.copy(df_generated), np.copy(train_generated), np.copy(test_generated), dep, quasi_identifiers_columns_ids_generated, quasi_identifiers_columns_types_generated,sensetive_columns_ids_generated,sensetive_columns_types_generated, predicted_column_generated,quasi_identifiers_boosting_ids_generated, sensitive_boosting_ids_generated,classification=classification_generated,generalization=True)
    print(f"\nОбезличивание датасета с caggle c k = {k}, t = {t}\n")
    method = GroupJoinGeneralization(quasi_identifiers_columns_types_caggle)
    metric = GroupJoinTCloseness(k, t, sensetive_columns_types_caggle, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_caggle = run_test_on_depersonalizer(np.copy(df_caggle), np.copy(train_caggle), np.copy(test_caggle), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle, quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle,classification=classification_caggle,generalization=True)
    print(f"\nОбезличивание короткого датасета c k = {k_short}, t = {t}\n")
    method = GroupJoinGeneralization(quasi_identifiers_columns_types_caggle[:2])
    metric = GroupJoinTCloseness(k_short, t, sensetive_columns_types_caggle, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True,generalization=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(pd.DataFrame([scores_generated, scores_caggle, scores_short], index, columns))

Обезличивание сгенерированного датасета c k = 2, t = 1

Время выполнения: 279.1313192844391 секунд
Максимальное k: 2
Максимальное l: 2
Минимальное t: 0.21032231086309605
Adversarial knowledge gain: 0.8497079999999809
Adversarial accuracy gain: 0.011000000000000003
Средний размер класса эквивалентности: 83.33333333333333
Distinctness: 0.012
Доля изменённых значений: 0.9988
Неоднородная энтропия: 3.0111106414728757
Потеря информации на основе энтропии 0.6978551255512174
My by element distance: 0.7439939272195769
Utility boosting score: 1.7334907387130372

Обезличивание датасета с caggle c k = 2, t = 1

Время выполнения: 143.84750723838806 секунд
Максимальное k: 2
Максимальное l: 1
Минимальное t: 0.6763598055104253
Adversarial knowledge gain: 0.9682209999999971
Adversarial accuracy gain: 0.40200000000000036
Средний размер класса эквивалентности: 2.7624309392265194
Distinctness: 0.362
Доля изменённых значений: 0.5434
Неоднородная энтропия: 8.40830553001823
Потеря информации на основе энтро

Показатели для k=2, k_short=2, t=1

In [264]:
scores_dfs[0]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,83.333333,0.012000,0.998800,3.011111,0.697855,0.743994,1.733491,2,2,0.210322,0.849708,0.011,279.131319
caggle,2.762431,0.362000,0.543400,8.408306,0.155435,0.078488,0.235000,2,1,0.676360,0.968221,0.402,143.847507
short,3.000000,0.333333,0.916667,1.918296,0.464905,0.141601,NaN,2,2,0.345144,0.722222,0.250,0.006000


Показатели для k=10, k_short=3, t=0.5

In [265]:
scores_dfs[1]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,100.000000,0.010,1.000,3.033918,0.695567,0.750485,1.990977,31,4,0.016842,0.858900,0.009000,279.734499
caggle,16.949153,0.059,0.685,5.727395,0.424717,0.373169,0.235000,10,2,0.327887,0.814692,0.127000,166.292962
short,4.000000,0.250,1.000,1.584963,0.557886,0.169378,NaN,4,3,0.171916,0.666667,0.166667,0.005000


Показатели для k=100, k_short=4, t=0.1

In [266]:
scores_dfs[2]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,200.000000,0.005000,1.0,2.249418,0.774286,0.710343,2.01954,141,4,0.005520,0.778184,0.004000,279.031330
caggle,333.333333,0.003000,1.0,1.057095,0.893821,0.621580,0.25500,106,33,0.027837,0.156738,0.005000,167.205945
short,6.000000,0.166667,1.0,1.000000,0.721057,0.301243,NaN,6,5,0.065160,0.500000,0.083333,0.004999


### Агрегация

#### K-анонимность

In [267]:
scores_dfs = []
for k, k_short in zip(k_s,k_s_short):
    print(f"Обезличивание сгенерированного датасета c k = {k}\n")
    method = GroupJoinAggregation(quasi_identifiers_columns_types_generated)
    metric = GroupJoinKAnonymity(k, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_generated = run_test_on_depersonalizer(np.copy(df_generated), np.copy(train_generated), np.copy(test_generated), dep, quasi_identifiers_columns_ids_generated, quasi_identifiers_columns_types_generated,sensetive_columns_ids_generated,sensetive_columns_types_generated, predicted_column_generated,quasi_identifiers_boosting_ids_generated, sensitive_boosting_ids_generated,classification=classification_generated,generalization=True)
    print(f"\nОбезличивание датасета с caggle c k = {k}\n")
    method = GroupJoinAggregation(quasi_identifiers_columns_types_caggle)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_caggle = run_test_on_depersonalizer(np.copy(df_caggle), np.copy(train_caggle), np.copy(test_caggle), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle, quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle,classification=classification_caggle,generalization=True)
    print(f"\nОбезличивание короткого датасета c k = {k_short}\n")
    method = GroupJoinAggregation(quasi_identifiers_columns_types_caggle[:2])
    metric = GroupJoinKAnonymity(k_short, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True,generalization=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(pd.DataFrame([scores_generated, scores_caggle, scores_short], index, columns))

Обезличивание сгенерированного датасета c k = 2

Время выполнения: 35.3481879234314 секунд
Максимальное k: 2
Максимальное l: 1
Минимальное t: 0.4249914906617438
Adversarial knowledge gain: 0.9965769999999741
Adversarial accuracy gain: 0.3150000000000002
Средний размер класса эквивалентности: 3.1545741324921135
Distinctness: 0.317
Доля изменённых значений: 0.4496
Неоднородная энтропия: 8.253412833761324
Потеря информации на основе энтропии 0.17182505681326032
My by element distance: 0.2239607726952682
Utility boosting score: 2.4934908023490854

Обезличивание датасета с caggle c k = 2

Время выполнения: 14.103745698928833 секунд
Максимальное k: 2
Максимальное l: 1
Минимальное t: 0.5822456666608443
Adversarial knowledge gain: 0.9721559999999975
Adversarial accuracy gain: 0.4350000000000004
Средний размер класса эквивалентности: 2.570694087403599
Distinctness: 0.389
Доля изменённых значений: 0.425
Неоднородная энтропия: 8.548242460572363
Потеря информации на основе энтропии 0.1413793011021

Показатели для k=2, k_short=2

In [268]:
scores_dfs[0]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,3.154574,0.317000,0.4496,8.253413,0.171825,0.223961,2.493491,2,1,0.424991,0.996577,0.315,35.348188
caggle,2.570694,0.389000,0.4250,8.548242,0.141379,0.042434,0.280000,2,1,0.582246,0.972156,0.435,14.103746
short,3.000000,0.333333,1.0000,1.959148,0.453510,0.062037,NaN,2,2,0.263780,0.736111,0.250,0.002001


Показатели для k=10, k_short=3

In [269]:
scores_dfs[1]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,13.513514,0.074,0.644333,6.157119,0.382174,0.377443,1.6677,10,2,0.261617,0.985325,0.124000,39.586035
caggle,16.393443,0.061,0.614000,5.870165,0.410376,0.078758,0.2400,10,3,0.285241,0.842148,0.151000,16.763073
short,4.000000,0.250,1.000000,1.554585,0.566359,0.092222,NaN,3,3,0.263780,0.652778,0.166667,0.002001


Показатели для k=100, k_short=4

In [270]:
scores_dfs[2]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,166.666667,0.006,0.796467,2.546505,0.744475,0.533444,2.08289,115,4,0.046721,0.824029,0.042000,39.802232
caggle,166.666667,0.006,0.934400,2.495829,0.749309,0.184024,0.25000,105,28,0.115500,0.339035,0.016000,16.803827
short,4.000000,0.250,1.000000,1.584963,0.557886,0.090278,NaN,4,3,0.171916,0.666667,0.166667,0.001999


#### L-разнообразие

In [271]:
scores_dfs = []
for k, k_short, l in zip(k_s,k_s_short,l_s):
    print(f"Обезличивание сгенерированного датасета c k = {k}, l = {l}\n")
    method = GroupJoinAggregation(quasi_identifiers_columns_types_generated)
    metric = GroupJoinLDiversity(k, l, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_generated = run_test_on_depersonalizer(np.copy(df_generated), np.copy(train_generated), np.copy(test_generated), dep, quasi_identifiers_columns_ids_generated, quasi_identifiers_columns_types_generated,sensetive_columns_ids_generated,sensetive_columns_types_generated, predicted_column_generated,quasi_identifiers_boosting_ids_generated, sensitive_boosting_ids_generated,classification=classification_generated,generalization=True)
    print(f"\nОбезличивание датасета с caggle c k = {k}, l = {l}\n")
    method = GroupJoinAggregation(quasi_identifiers_columns_types_caggle)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_caggle = run_test_on_depersonalizer(np.copy(df_caggle), np.copy(train_caggle), np.copy(test_caggle), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle, quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle,classification=classification_caggle,generalization=True)
    print(f"\nОбезличивание короткого датасета c k = {k_short}, l = {l}\n")
    method = GroupJoinAggregation(quasi_identifiers_columns_types_caggle[:2])
    metric = GroupJoinLDiversity(k_short, l, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True,generalization=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(pd.DataFrame([scores_generated, scores_caggle, scores_short], index, columns))

Обезличивание сгенерированного датасета c k = 2, l = 2

Время выполнения: 37.89886665344238 секунд
Максимальное k: 2
Максимальное l: 2
Минимальное t: 0.4309021300891787
Adversarial knowledge gain: 0.9963689999999735
Adversarial accuracy gain: 0.2990000000000002
Средний размер класса эквивалентности: 3.3222591362126246
Distinctness: 0.301
Доля изменённых значений: 0.461
Неоднородная энтропия: 8.173390462938103
Потеря информации на основе энтропии 0.17985476812723888
My by element distance: 0.23041365564055216
Utility boosting score: 1.7969400987833195

Обезличивание датасета с caggle c k = 2, l = 2

Время выполнения: 18.270965576171875 секунд
Максимальное k: 2
Максимальное l: 2
Минимальное t: 0.5787123333275462
Adversarial knowledge gain: 0.9491299999999967
Adversarial accuracy gain: 0.3310000000000003
Средний размер класса эквивалентности: 4.237288135593221
Distinctness: 0.236
Доля изменённых значений: 0.5144
Неоднородная энтропия: 7.7387017157021125
Потеря информации на основе энтропи

Показатели для k=2, k_short=2, l=2

In [272]:
scores_dfs[0]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,3.322259,0.301000,0.4610,8.173390,0.179855,0.230414,1.79694,2,2,0.430902,0.996369,0.299,37.898867
caggle,4.237288,0.236000,0.5144,7.738702,0.222693,0.052476,0.30500,2,2,0.578712,0.949130,0.331,18.270966
short,3.000000,0.333333,1.0000,1.959148,0.453510,0.062037,NaN,2,2,0.263780,0.736111,0.250,0.002000


Показатели для k=10, k_short=3, l=3

In [273]:
scores_dfs[1]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,15.873016,0.063,0.648867,5.911758,0.406795,0.382989,2.108844,10,4,0.217114,0.982532,0.099000,41.536790
caggle,15.151515,0.066,0.644200,5.981451,0.399198,0.076358,0.245000,10,3,0.311107,0.851944,0.173000,19.590099
short,4.000000,0.250,1.000000,1.554585,0.566359,0.092222,NaN,3,3,0.263780,0.652778,0.166667,0.002000


Показатели для k=100, k_short=4, l=4

In [274]:
scores_dfs[2]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,166.666667,0.006000,0.792133,2.468950,0.752257,0.527398,1.988755,107,4,0.027713,0.802863,0.019000,42.678277
caggle,142.857143,0.007000,0.915000,2.782201,0.720544,0.170705,0.245000,115,35,0.101486,0.398839,0.019000,19.878000
short,6.000000,0.166667,1.000000,0.918296,0.743848,0.104514,NaN,4,4,0.171916,0.444444,0.083333,0.001000


#### T-близость

In [275]:
scores_dfs = []
for k, k_short, t in zip(k_s,k_s_short,t_s):
    print(f"Обезличивание сгенерированного датасета c k = {k}, t = {t}\n")
    method = GroupJoinAggregation(quasi_identifiers_columns_types_generated)
    metric = GroupJoinTCloseness(k, t, sensetive_columns_types_generated, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_generated = run_test_on_depersonalizer(np.copy(df_generated), np.copy(train_generated), np.copy(test_generated), dep, quasi_identifiers_columns_ids_generated, quasi_identifiers_columns_types_generated,sensetive_columns_ids_generated,sensetive_columns_types_generated, predicted_column_generated,quasi_identifiers_boosting_ids_generated, sensitive_boosting_ids_generated,classification=classification_generated,generalization=True)
    print(f"\nОбезличивание датасета с caggle c k = {k}, t = {t}\n")
    method = GroupJoinAggregation(quasi_identifiers_columns_types_caggle)
    metric = GroupJoinTCloseness(k, t, sensetive_columns_types_caggle, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_caggle = run_test_on_depersonalizer(np.copy(df_caggle), np.copy(train_caggle), np.copy(test_caggle), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle, quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle,classification=classification_caggle,generalization=True)
    print(f"\nОбезличивание короткого датасета c k = {k_short}, t = {t}\n")
    method = GroupJoinAggregation(quasi_identifiers_columns_types_caggle[:2])
    metric = GroupJoinTCloseness(k_short, t, sensetive_columns_types_caggle, seed=seed)
    dep = GroupJoinDepersonalizator(method, metric)
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True,generalization=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(pd.DataFrame([scores_generated, scores_caggle, scores_short], index, columns))

Обезличивание сгенерированного датасета c k = 2, t = 1

Время выполнения: 540.4233121871948 секунд
Максимальное k: 2
Максимальное l: 2
Минимальное t: 0.21909113614787656
Adversarial knowledge gain: 0.8494389999999803
Adversarial accuracy gain: 0.017000000000000008
Средний размер класса эквивалентности: 52.63157894736842
Distinctness: 0.019
Доля изменённых значений: 0.8262666666666667
Неоднородная энтропия: 3.135897425640411
Потеря информации на основе энтропии 0.685333603852259
My by element distance: 0.5759286435210474
Utility boosting score: 2.1386636796386664

Обезличивание датасета с caggle c k = 2, t = 1

Время выполнения: 293.3978343009949 секунд
Максимальное k: 2
Максимальное l: 1
Минимальное t: 0.6763598055104253
Adversarial knowledge gain: 0.9653159999999973
Adversarial accuracy gain: 0.37700000000000033
Средний размер класса эквивалентности: 2.9585798816568047
Distinctness: 0.338
Доля изменённых значений: 0.4506
Неоднородная энтропия: 8.291741930356983
Потеря информации на ос

Показатели для k=2, k_short=2, t=1

In [276]:
scores_dfs[0]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,52.631579,0.019000,0.826267,3.135897,0.685334,0.575929,2.138664,2,2,0.219091,0.849439,0.017,540.423312
caggle,2.958580,0.338000,0.450600,8.291742,0.167143,0.046231,0.265000,2,1,0.676360,0.965316,0.377,293.397834
short,3.000000,0.333333,0.916667,1.918296,0.464905,0.065278,NaN,2,2,0.345144,0.722222,0.250,0.015999


Показатели для k=10, k_short=3, t=0.5

In [277]:
scores_dfs[1]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,100.000000,0.010,0.8340,2.686239,0.730454,0.583053,2.130476,12,4,0.036887,0.808397,0.008000,562.214049
caggle,17.857143,0.056,0.7332,5.473510,0.450218,0.112757,0.245000,10,1,0.278183,0.763791,0.118000,174.404515
short,4.000000,0.250,1.0000,1.584963,0.557886,0.090278,NaN,4,3,0.171916,0.666667,0.166667,0.005998


Показатели для k=100, k_short=4, t=0.1

In [278]:
scores_dfs[2]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,333.333333,0.003000,0.847733,1.562598,0.843204,0.596921,2.041764,261,4,0.00337,0.655773,0.002000,486.790952
caggle,333.333333,0.003000,0.999200,1.507548,0.848576,0.257461,0.245000,188,54,0.01101,0.192408,0.006000,174.767231
short,6.000000,0.166667,1.000000,1.000000,0.721057,0.189815,NaN,6,5,0.06516,0.500000,0.083333,0.018000


## Алгоритм Datafly

In [279]:
scores_dfs = []
for k, k_short in zip(k_s,k_s_short):
    print(f"Обезличивание сгенерированного датасета c k = {k}\n")
    dep = Datafly(k, quasi_identifiers_columns_types_generated, k_suppressed_lines=0)
    scores_generated = run_test_on_depersonalizer(np.copy(df_generated), np.copy(train_generated), np.copy(test_generated), dep, quasi_identifiers_columns_ids_generated, quasi_identifiers_columns_types_generated,sensetive_columns_ids_generated,sensetive_columns_types_generated, predicted_column_generated,quasi_identifiers_boosting_ids_generated, sensitive_boosting_ids_generated,classification=classification_generated, generalization=True)
    print(f"\nОбезличивание датасета с caggle c k = {k}\n")
    dep = Datafly(k, quasi_identifiers_columns_types_caggle, k_suppressed_lines=0)
    scores_caggle = run_test_on_depersonalizer(np.copy(df_caggle), np.copy(train_caggle), np.copy(test_caggle), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle, quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle,classification=classification_caggle, generalization=True)
    print(f"\nОбезличивание короткого датасета c k = {k_short}\n")
    dep = Datafly(k_short, quasi_identifiers_columns_types_caggle[:2], k_suppressed_lines=0)
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True, generalization=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(pd.DataFrame([scores_generated, scores_caggle, scores_short], index, columns))

Обезличивание сгенерированного датасета c k = 2

Время выполнения: 13.398889303207397 секунд
Максимальное k: 2
Максимальное l: 2
Минимальное t: 0.27055376642975393
Adversarial knowledge gain: 0.9605839999999751
Adversarial accuracy gain: 0.08000000000000014
Средний размер класса эквивалентности: 25.0
Distinctness: 0.04
Доля изменённых значений: 0.9580666666666666
Неоднородная энтропия: 4.882181132696284
Потеря информации на основе энтропии 0.5101056782645554
My by element distance: 0.58402494786979
Utility boosting score: 1.8898301738781924

Обезличивание датасета с caggle c k = 2

Время выполнения: 2.138051986694336 секунд
Максимальное k: 19
Максимальное l: 5
Минимальное t: 0.1182223833695315
Adversarial knowledge gain: 0.5322639999999994
Adversarial accuracy gain: 0.041
Средний размер класса эквивалентности: 62.5
Distinctness: 0.016
Доля изменённых значений: 0.8578
Неоднородная энтропия: 3.715139937378223
Потеря информации на основе энтропии 0.6268360350975282
My by element distance:

Показатели для k=2, k_short=2

In [280]:
scores_dfs[0]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,25.0,0.040000,0.958067,4.882181,0.510106,0.584025,1.88983,2,2,0.270554,0.960584,0.080000,13.398889
caggle,62.5,0.016000,0.857800,3.715140,0.626836,0.204499,0.24500,19,5,0.118222,0.532264,0.041000,2.138052
short,6.0,0.166667,1.000000,0.979869,0.726673,0.253632,NaN,5,5,0.193963,0.486111,0.083333,0.001000


Показатели для k=10, k_short=3

In [281]:
scores_dfs[1]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,62.5,0.016000,0.977267,3.864400,0.612233,0.607885,1.993251,22,4,0.109122,0.926033,0.052000,13.301902
caggle,62.5,0.016000,0.857800,3.715140,0.626836,0.204499,0.245000,19,5,0.118222,0.532264,0.041000,2.163972
short,6.0,0.166667,1.000000,0.979869,0.726673,0.253632,NaN,5,5,0.193963,0.486111,0.083333,0.001007


Показатели для k=100, k_short=4

In [282]:
scores_dfs[2]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,250.0,0.004000,1.0000,1.971446,0.802178,0.637073,1.870775,196,4,0.030425,0.739146,0.016000,13.330360
caggle,250.0,0.004000,0.8578,1.843812,0.814800,0.263703,0.245000,128,30,0.091030,0.276633,0.014000,2.197001
short,6.0,0.166667,1.0000,0.979869,0.726673,0.253632,NaN,5,5,0.193963,0.486111,0.083333,0.002007


## Алгоритм разбиения столбцов на группы с равным минимальными размером

In [283]:
scores_dfs = []
for k, k_short in zip(k_s,k_s_short):
    print(f"Обезличивание сгенерированного датасета c k = {k}\n")
    dep = GeneralizationGreedyByOneEqualSizedGroups(k, quasi_identifiers_columns_types_generated)
    scores_generated = run_test_on_depersonalizer(np.copy(df_generated), np.copy(train_generated), np.copy(test_generated), dep, quasi_identifiers_columns_ids_generated, quasi_identifiers_columns_types_generated,sensetive_columns_ids_generated,sensetive_columns_types_generated, predicted_column_generated,quasi_identifiers_boosting_ids_generated, sensitive_boosting_ids_generated,classification=classification_generated, generalization=True)
    print(f"\nОбезличивание датасета с caggle c k = {k}\n")
    dep = GeneralizationGreedyByOneEqualSizedGroups(k, quasi_identifiers_columns_types_caggle)
    scores_caggle = run_test_on_depersonalizer(np.copy(df_caggle), np.copy(train_caggle), np.copy(test_caggle), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle, quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle,classification=classification_caggle, generalization=True)
    print(f"\nОбезличивание короткого датасета c k = {k_short}\n")
    dep = GeneralizationGreedyByOneEqualSizedGroups(k_short, quasi_identifiers_columns_types_caggle[:2])
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle,sensetive_columns_ids_caggle,sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True, generalization=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(pd.DataFrame([scores_generated, scores_caggle, scores_short], index, columns))

Обезличивание сгенерированного датасета c k = 2

Время выполнения: 59.19961881637573 секунд
Максимальное k: 23
Максимальное l: 4
Минимальное t: 0.13748022560871523
Adversarial knowledge gain: 0.9678279999999745
Adversarial accuracy gain: 0.08300000000000007
Средний размер класса эквивалентности: 31.25
Distinctness: 0.032
Доля изменённых значений: 0.9333333333333333
Неоднородная энтропия: 4.983394952219843
Потеря информации на основе энтропии 0.4999495463804514
My by element distance: 0.5880922686507213
Utility boosting score: 2.0037945643802835

Обезличивание датасета с caggle c k = 2

Время выполнения: 13.470199346542358 секунд
Максимальное k: 5
Максимальное l: 2
Минимальное t: 0.15844670050753382
Adversarial knowledge gain: 0.42532200000000003
Adversarial accuracy gain: 0.022000000000000013
Средний размер класса эквивалентности: 83.33333333333333
Distinctness: 0.012
Доля изменённых значений: 1.0
Неоднородная энтропия: 3.125886029128833
Потеря информации на основе энтропии 0.686023125

Показатели для k=2, k_short=2

In [284]:
scores_dfs[0]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,31.250000,0.032000,0.933333,4.983395,0.499950,0.588092,2.003795,23,4,0.137480,0.967828,0.083,59.199619
caggle,83.333333,0.012000,1.000000,3.125886,0.686023,0.217594,0.245000,5,2,0.158447,0.425322,0.022,13.470199
short,3.000000,0.333333,1.000000,2.000000,0.442114,0.080365,NaN,3,3,0.263780,0.750000,0.250,0.001000


Показатели для k=10, k_short=3

In [285]:
scores_dfs[1]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,31.25,0.032000,0.933333,4.983395,0.499950,0.588092,2.003795,23,4,0.137480,0.967828,0.083,58.314616
caggle,500.00,0.002000,1.000000,0.999928,0.899563,0.326796,0.245000,495,77,0.006325,0.151440,0.006,9.968015
short,3.00,0.333333,1.000000,2.000000,0.442114,0.080365,NaN,3,3,0.263780,0.750000,0.250,0.000000


Показатели для k=100, k_short=4

In [286]:
scores_dfs[2]

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
generated,125.0,0.008000,0.933333,2.998395,0.699131,0.615888,1.931402,114,4,0.034748,0.874214,0.032000,48.648113
caggle,500.0,0.002000,1.000000,0.999928,0.899563,0.326796,0.245000,495,77,0.006325,0.151440,0.006000,8.185860
short,6.0,0.166667,1.000000,1.000000,0.721057,0.165549,NaN,6,6,0.190289,0.500000,0.083333,0.000000


## Рандомизация

In [287]:
def get_randomization_scores(df, dep, qi_ids, qi_types, sensitives_ids, sensitives_types,predicted_column,quasi_identifiers_boosting_ids, sensitive_boosting_ids,short = False,classification=True):
    start_time = time.time()
    depersonalized_df = dep.depersonalize(df, identifiers_ids=[], quasi_identifiers_ids=qi_ids, sensitives_ids=sensitives_ids)[0]
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Время выполнения: {elapsed_time} секунд")
    
    my_by_element_distance_score = my_by_element_distance(df[:,qi_ids], depersonalized_df[:,qi_ids], qi_types)
    print(f"My by element distance: {my_by_element_distance_score}")
    
    train, test = train_test_split(df, test_size=0.2, random_state=seed, shuffle=False)
    train_depersonalized, test_depersonalized = train_test_split(depersonalized_df, test_size=0.2, random_state=seed, shuffle=False)
    
    if not short:
        quasi_identifiers_columns_types_boosting = []
        sensitive_columns_types_boosting = []
        for i in quasi_identifiers_boosting_ids:
            quasi_identifiers_columns_types_boosting.append(qi_types[qi_ids.index(i)])
        for i in sensitive_boosting_ids:
            sensitive_columns_types_boosting.append(sensitives_types[sensitives_ids.index(i)])
            
        train_initial, test_initial = prepare_data(train[:,quasi_identifiers_boosting_ids], test[:,quasi_identifiers_boosting_ids], [0] * len(quasi_identifiers_boosting_ids) , quasi_identifiers_columns_types_boosting, normalize=True)
        train_depersonalized_sec, test_depersonalized_sec = prepare_data(train_depersonalized[:,quasi_identifiers_boosting_ids + sensitive_boosting_ids], test_depersonalized[:,quasi_identifiers_boosting_ids + sensitive_boosting_ids], [0] * len(quasi_identifiers_boosting_ids) + [0]*len(sensitive_boosting_ids), quasi_identifiers_columns_types_boosting+sensitive_columns_types_boosting, normalize=True)
        
        security_boosting_score = get_boosting_security_score(train_initial, train_depersonalized_sec, test_initial, test_depersonalized_sec)
        
    print(f"Security boosting score: {security_boosting_score}")
        
    if not short:
        parameters = {
            #'n_estimators':list(range(10, 201, 15)),
            #'alpha':[i / 10 for i in range(1, 10, 4)],
            'min_samples_leaf':[i for i in range(1, 6)],
            'criterion':['squared_error'],
        }
        quasi_identifiers_columns_types_boosting = quasi_identifiers_columns_types_boosting.copy()
        quasi_identifiers_boosting_ids = quasi_identifiers_boosting_ids.copy()
        sensitive_columns_types_boosting = sensitive_columns_types_boosting.copy()
        sensitive_boosting_ids = sensitive_boosting_ids.copy()
        if predicted_column in quasi_identifiers_boosting_ids:
            quasi_identifiers_columns_types_boosting.pop(quasi_identifiers_boosting_ids.index(predicted_column))
            quasi_identifiers_boosting_ids.remove(predicted_column)
        if predicted_column in sensitive_boosting_ids:
            sensitive_columns_types_boosting.pop(sensitive_boosting_ids.index(predicted_column))
            sensitive_boosting_ids.remove(predicted_column)
        
        train_depersonalized, test_depersonalized = prepare_data(train_depersonalized[:,quasi_identifiers_boosting_ids + sensitive_boosting_ids], test_depersonalized[:,quasi_identifiers_boosting_ids + sensitive_boosting_ids], [0] * len(quasi_identifiers_boosting_ids) + [0]*len(sensitive_boosting_ids), quasi_identifiers_columns_types_boosting+sensitive_columns_types_boosting, normalize=True)
        y_train = train[:,predicted_column]
        y_test = test[:,predicted_column]
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=ConvergenceWarning)
            if classification:
                reg = GridSearchCV(GradientBoostingClassifier(random_state=seed,n_estimators=200), parameters)
                reg.fit(train_depersonalized, y_train)
                res = reg.predict(test_depersonalized)
                #utility_boosting_score = ((res - y_test)**2).mean()**0.5
                utility_boosting_score = accuracy_score(y_test, res)
            else:
                reg = GridSearchCV(GradientBoostingRegressor(random_state=seed,n_estimators=200), parameters)
                reg.fit(train_depersonalized, y_train)
                res = reg.predict(test_depersonalized)
                utility_boosting_score = mean_absolute_percentage_error(y_test, res)#((res - y_test)**2).mean()**0.5
            print(f"Utility boosting score: {utility_boosting_score}")
        
        return [utility_boosting_score, my_by_element_distance_score, security_boosting_score, elapsed_time]

In [288]:
scores_dfs = []
columns_rand = ['Utility boosting score', 'My by element distance', 'Security boosting score', 'Время выполнения, секунд']
quasi_identifiers_columns_types_caggle = ['real','real','real','real','real']
for scale in scales:
    print(f"Обезличивание сгенерированного датасета c scale = {scale}\n")
    dep =  RandomizationBaselineDepersonalizator(quasi_identifiers_types=quasi_identifiers_columns_types_generated, scale = scale)
    scores_generated = get_randomization_scores(df_generated, dep, quasi_identifiers_columns_ids_generated, quasi_identifiers_columns_types_generated, sensetive_columns_ids_generated, sensetive_columns_types_generated,predicted_column_generated,quasi_identifiers_boosting_ids_generated,sensitive_boosting_ids_generated,classification=classification_generated)
    
    print(f"\nОбезличивание датасета с caggle c scale = {scale}\n")
    dep =  RandomizationBaselineDepersonalizator(quasi_identifiers_types=quasi_identifiers_columns_types_caggle, scale = scale)
    scores_caggle = get_randomization_scores(df_caggle, dep, quasi_identifiers_columns_ids_caggle, quasi_identifiers_columns_types_caggle, sensetive_columns_ids_caggle, sensetive_columns_types_caggle, predicted_column_caggle, quasi_identifiers_boosting_ids_caggle,sensitive_boosting_ids_caggle,classification=classification_caggle)
    
    print("\n\n======================================================\n\n")
    scores_dfs.append(pd.DataFrame([scores_generated, scores_caggle], ['generated', 'caggle'], columns_rand))

Обезличивание сгенерированного датасета c scale = 0.05

Время выполнения: 0.01799941062927246 секунд
My by element distance: 0.28002322005530994
Security boosting score: 1.0
Utility boosting score: 1.636219034011544

Обезличивание датасета с caggle c scale = 0.05

Время выполнения: 0.011999368667602539 секунд
My by element distance: 0.05090442676486029
Security boosting score: 0.6599999999999999
Utility boosting score: 0.505




Обезличивание сгенерированного датасета c scale = 0.25

Время выполнения: 0.03899836540222168 секунд
My by element distance: 0.31807191561266895
Security boosting score: 1.0
Utility boosting score: 1.744066732718195

Обезличивание датасета с caggle c scale = 0.25

Время выполнения: 0.027001142501831055 секунд
My by element distance: 0.13200515173791044
Security boosting score: 0.915
Utility boosting score: 0.375




Обезличивание сгенерированного датасета c scale = 1

Время выполнения: 0.018000125885009766 секунд
My by element distance: 0.38294343250734175
Secu

Показатели для sсale = 0.05

In [289]:
scores_dfs[0]

,Utility boosting score,My by element distance,Security boosting score,"Время выполнения, секунд"
generated,1.636219,0.280023,1.00,0.017999
caggle,0.505000,0.050904,0.66,0.011999


Показатели для sсale = 0.25

In [290]:
scores_dfs[1]

,Utility boosting score,My by element distance,Security boosting score,"Время выполнения, секунд"
generated,1.744067,0.318072,1.000,0.038998
caggle,0.375000,0.132005,0.915,0.027001


Показатели для sсale = 1

In [291]:
scores_dfs[2]

,Utility boosting score,My by element distance,Security boosting score,"Время выполнения, секунд"
generated,1.511333,0.382943,1.0,0.018
caggle,0.390000,0.287936,1.0,0.026


## Алгоритм перебора с подавлением

#### K-анонимность

In [189]:
columns = ["Средний размер класса эквивалентности", "Distinctness", "Доля изменённых значений", "Неоднородная энтропия", "Потеря информации на основе энтропии", "My by element distance", "Utility boosting score", "max k", "max l", "min t", "Adversarial knowledge gain", "Adversarial accuracy gain", "Время выполнения, секунд"]
scores_dfs = []
for k_short in k_s_short:
    print(f"\nОбезличивание короткого датасета c k = {k_short}\n")
    dep = SuppressionKAnonymityBaseline(k_short)
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, [0,1 ], quasi_identifiers_columns_types_caggle[:2],[2,3,4],sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(scores_short)


Обезличивание короткого датасета c k = 2

Время выполнения: 125.77111387252808 секунд
Максимальное k: 2
Максимальное l: 2
Минимальное t: 0.2921099290778588
Adversarial knowledge gain: 0.27777777777777785
Adversarial accuracy gain: 0.08333333333333333
Средний размер класса эквивалентности: 6.0
Distinctness: 0.16666666666666666
Доля изменённых значений: 0.9166666666666666
Неоднородная энтропия: 0.6500224216483541
Потеря информации на основе энтропии 0.8186808309661274
My by element distance: 0.9166666666666667





Обезличивание короткого датасета c k = 3

Время выполнения: 136.59828543663025 секунд
Максимальное k: 12
Максимальное l: 9
Минимальное t: 0
Adversarial knowledge gain: 0.0
Adversarial accuracy gain: 0.0
Средний размер класса эквивалентности: 12.0
Distinctness: 0.08333333333333333
Доля изменённых значений: 1.0
Неоднородная энтропия: 0.0
Потеря информации на основе энтропии 1.0
My by element distance: 1.0





Обезличивание короткого датасета c k = 4

Время выполнения: 135.3713

In [190]:
pd.DataFrame(scores_dfs, ['k=2','k=3','k=4'], columns=columns)

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
k=2,6.0,0.166667,0.916667,0.650022,0.818681,0.916667,None,2,2,0.29211,0.277778,0.083333,125.771114
k=3,12.0,0.083333,1.000000,0.000000,1.000000,1.000000,None,12,9,0.00000,0.000000,0.000000,136.598285
k=4,12.0,0.083333,1.000000,0.000000,1.000000,1.000000,None,12,9,0.00000,0.000000,0.000000,135.371378


#### L-разнообразие

In [191]:
scores_dfs = []
for k_short, l in zip(k_s_short,l_s):
    print(f"\nОбезличивание короткого датасета c k = {k_short}, l = {l}\n")
    dep = SuppressionLDiversityBaseline(k_short, l)
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, [0,1 ], quasi_identifiers_columns_types_caggle[:2],[2,3,4],sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(scores_short)


Обезличивание короткого датасета c k = 2, l = 2

Время выполнения: 343.3185877799988 секунд
Максимальное k: 2
Максимальное l: 2
Минимальное t: 0.2921099290778588
Adversarial knowledge gain: 0.27777777777777785
Adversarial accuracy gain: 0.08333333333333333
Средний размер класса эквивалентности: 6.0
Distinctness: 0.16666666666666666
Доля изменённых значений: 0.9166666666666666
Неоднородная энтропия: 0.6500224216483541
Потеря информации на основе энтропии 0.8186808309661274
My by element distance: 0.9166666666666667





Обезличивание короткого датасета c k = 3, l = 3

Время выполнения: 321.17678904533386 секунд
Максимальное k: 12
Максимальное l: 9
Минимальное t: 0
Adversarial knowledge gain: 0.0
Adversarial accuracy gain: 0.0
Средний размер класса эквивалентности: 12.0
Distinctness: 0.08333333333333333
Доля изменённых значений: 1.0
Неоднородная энтропия: 0.0
Потеря информации на основе энтропии 1.0
My by element distance: 1.0





Обезличивание короткого датасета c k = 4, l = 4

Время 

In [192]:
pd.DataFrame(scores_dfs, ['k=2,l=2','k=3,l=3','k=4,l=4'], columns=columns)

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
"k=2,l=2",6.0,0.166667,0.916667,0.650022,0.818681,0.916667,None,2,2,0.29211,0.277778,0.083333,343.318588
"k=3,l=3",12.0,0.083333,1.000000,0.000000,1.000000,1.000000,None,12,9,0.00000,0.000000,0.000000,321.176789
"k=4,l=4",12.0,0.083333,1.000000,0.000000,1.000000,1.000000,None,12,9,0.00000,0.000000,0.000000,233.820024


#### T-близость

In [193]:
scores_dfs = []
for k_short, t in zip(k_s_short,t_s):
    print(f"\nОбезличивание короткого датасета c k = {k_short}, t = {t}\n")
    dep = SuppressionTClosenessBaseline(k_short, t, sensetive_columns_types_caggle)
    scores_short = run_test_on_depersonalizer(np.copy(df_short), np.copy(train_short), np.copy(test_short), dep, [0,1 ], quasi_identifiers_columns_types_caggle[:2],[2,3,4],sensetive_columns_types_caggle, predicted_column_caggle,quasi_identifiers_boosting_ids_caggle, sensitive_boosting_ids_caggle, short=True)
    print("\n\n======================================================\n\n")
    scores_dfs.append(scores_short)


Обезличивание короткого датасета c k = 2, t = 1

Время выполнения: 466.2951204776764 секунд
Максимальное k: 2
Максимальное l: 2
Минимальное t: 0.2921099290778588
Adversarial knowledge gain: 0.27777777777777785
Adversarial accuracy gain: 0.08333333333333333
Средний размер класса эквивалентности: 6.0
Distinctness: 0.16666666666666666
Доля изменённых значений: 0.9166666666666666
Неоднородная энтропия: 0.6500224216483541
Потеря информации на основе энтропии 0.8186808309661274
My by element distance: 0.9166666666666667





Обезличивание короткого датасета c k = 3, t = 0.5

Время выполнения: 352.2959551811218 секунд
Максимальное k: 12
Максимальное l: 9
Минимальное t: 0
Adversarial knowledge gain: 0.0
Adversarial accuracy gain: 0.0
Средний размер класса эквивалентности: 12.0
Distinctness: 0.08333333333333333
Доля изменённых значений: 1.0
Неоднородная энтропия: 0.0
Потеря информации на основе энтропии 1.0
My by element distance: 1.0





Обезличивание короткого датасета c k = 4, t = 0.1

Вре

In [194]:
pd.DataFrame(scores_dfs, ['k=2,t=1','k=3,t=0.5','k=4,t=0.1'], columns=columns)

,Средний размер класса эквивалентности,Distinctness,Доля изменённых значений,Неоднородная энтропия,Потеря информации на основе энтропии,My by element distance,Utility boosting score,max k,max l,min t,Adversarial knowledge gain,Adversarial accuracy gain,"Время выполнения, секунд"
"k=2,t=1",6.0,0.166667,0.916667,0.650022,0.818681,0.916667,None,2,2,0.29211,0.277778,0.083333,466.295120
"k=3,t=0.5",12.0,0.083333,1.000000,0.000000,1.000000,1.000000,None,12,9,0.00000,0.000000,0.000000,352.295955
"k=4,t=0.1",12.0,0.083333,1.000000,0.000000,1.000000,1.000000,None,12,9,0.00000,0.000000,0.000000,237.167198


# Демонстрация работы алгоритмов для специальных задач

## Алгоритм объединения распределённого по горизонтали датасета с сохранением конфиденциальности за счёт обезличивания

In [17]:
df_caggle = pd.read_csv("Bank_Personal_Loan_Modelling.csv", keep_default_na=False)
df_caggle = df_caggle.to_numpy()[:1000]
df = np.copy(df_caggle)

df = np.delete(df, (0), axis=1)
dep1 = DistributedDataOwnerKAnonymityDepersonalizator(df[:,[0,1,2]], df[:,[5,7,8,9]], 2, 7, seed=seed, quasi_identifiers_types=['real'] * 3)
dep2 = DistributedDataOwnerKAnonymityDepersonalizator(df[:,[3,4,6]], df[:,[10,11,12]], 2, 8, seed=seed, quasi_identifiers_types=['real'] * 2 + ['ordered'])
dep1.set_other_data_owner(dep2)
dep2.set_other_data_owner(dep1)
dep1.exchange_data()
pd.DataFrame(dep1.joined_data,columns = ['Age','Experience','Income','CCAvg','Mortgage','Personal Loan','Securities Account','ZIP Code','Family','Education','CD Account','Online','CreditCard'])

,Age,Experience,Income,CCAvg,Mortgage,Personal Loan,Securities Account,ZIP Code,Family,Education,CD Account,Online,CreditCard
0,30.53112,5.232365,62.327801,1.6,0.0,0.0,1.0,89147.741379,4.0,2.0,0.0,0.0,0.0
1,41.20202,16.090909,90.488215,1.5,0.0,0.0,1.0,90258.62963,3.0,2.0,0.0,0.0,0.0
2,41.20202,16.090909,90.488215,1.0,0.0,0.0,0.0,94665.463415,1.0,1.0,0.0,0.0,0.0
3,41.20202,16.090909,90.488215,2.7,0.0,0.0,0.0,94114.219512,1.0,2.0,0.0,0.0,0.0
4,30.53112,5.232365,62.327801,1.0,0.0,0.0,0.0,89147.741379,4.0,2.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,30.53112,5.232365,62.327801,1.6,0.0,0.0,0.0,94408.337662,2.0,1.0,0.0,1.0,1.0
996,30.53112,5.232365,62.327801,1.67,214.0,0.0,0.0,92117.390244,2.0,1.0,0.0,0.0,0.0
997,41.20202,16.090909,90.488215,2.1,0.0,0.0,0.0,92162.896552,3.0,2.0,0.0,0.0,0.0
998,54.193069,28.841584,99.876238,2.8,333.0,0.0,0.0,93001.225,1.0,2.0,0.0,1.0,0.0


## Хеширование идентификаторов

In [18]:
df = pd.read_csv("with_identifiers.csv", keep_default_na=False)
columns = df.columns
identifiers_ids = [0, 1, 2, 6, 7, 8, 12]
hashed_df = IdentifierHasher().depersonalize(df, identifiers_ids=identifiers_ids)[0]
pd.DataFrame(hashed_df, columns=columns)

,id,first_name,last_name,gender,birth_date,age,email,phone_number,address,city,...,loan_purpose,credit_score,education_level,employment_status,years_of_experience,family_size,number_of_children,marital_status,housing_status,industry
0,fb19451fdc9b509b753dcf3d406adf11ab75980f7ee441...,fb19451fdc9b509b753dcf3d406adf11ab75980f7ee441...,fb19451fdc9b509b753dcf3d406adf11ab75980f7ee441...,Female,1991-08-03,34,fb19451fdc9b509b753dcf3d406adf11ab75980f7ee441...,fb19451fdc9b509b753dcf3d406adf11ab75980f7ee441...,fb19451fdc9b509b753dcf3d406adf11ab75980f7ee441...,Bryanside,...,Car,666,Master's,Employed,1,2,1,Divorced,Living with Family,Transportation
1,a0e7ec80537c104dde524a484a4068fa38a8aa7444ac89...,a0e7ec80537c104dde524a484a4068fa38a8aa7444ac89...,a0e7ec80537c104dde524a484a4068fa38a8aa7444ac89...,Male,2001-07-10,24,a0e7ec80537c104dde524a484a4068fa38a8aa7444ac89...,a0e7ec80537c104dde524a484a4068fa38a8aa7444ac89...,a0e7ec80537c104dde524a484a4068fa38a8aa7444ac89...,Seanfurt,...,Business,783,Associate Degree,Unemployed,0,1,0,Single,Mortgage,Hospitality
2,4a01d8f748b7fe9a771b4b166a8aeab5eb8a8b0b79d192...,4a01d8f748b7fe9a771b4b166a8aeab5eb8a8b0b79d192...,4a01d8f748b7fe9a771b4b166a8aeab5eb8a8b0b79d192...,Female,1969-09-28,56,4a01d8f748b7fe9a771b4b166a8aeab5eb8a8b0b79d192...,4a01d8f748b7fe9a771b4b166a8aeab5eb8a8b0b79d192...,4a01d8f748b7fe9a771b4b166a8aeab5eb8a8b0b79d192...,Davebury,...,Car,641,Bachelor's,Student,35,4,3,Single,Rent,Finance
3,6980f7fdd7992e2a2ec8e19d41862fb9ef701191b868a6...,6980f7fdd7992e2a2ec8e19d41862fb9ef701191b868a6...,6980f7fdd7992e2a2ec8e19d41862fb9ef701191b868a6...,Male,1954-04-08,71,6980f7fdd7992e2a2ec8e19d41862fb9ef701191b868a6...,6980f7fdd7992e2a2ec8e19d41862fb9ef701191b868a6...,6980f7fdd7992e2a2ec8e19d41862fb9ef701191b868a6...,Chadland,...,Business,598,PhD,Student,28,5,4,Widowed,Living with Family,Hospitality
4,a7babe25ff34330376b3c8e3388e85780e15187db6b283...,a7babe25ff34330376b3c8e3388e85780e15187db6b283...,a7babe25ff34330376b3c8e3388e85780e15187db6b283...,Male,1948-01-31,77,a7babe25ff34330376b3c8e3388e85780e15187db6b283...,a7babe25ff34330376b3c8e3388e85780e15187db6b283...,a7babe25ff34330376b3c8e3388e85780e15187db6b283...,New Hannah,...,Personal,491,PhD,Unemployed,51,4,2,Married,Rent,IT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,65fa81cf55f9b067e446b1a60510ebf2f449fa973105bd...,65fa81cf55f9b067e446b1a60510ebf2f449fa973105bd...,65fa81cf55f9b067e446b1a60510ebf2f449fa973105bd...,Female,2002-01-27,23,65fa81cf55f9b067e446b1a60510ebf2f449fa973105bd...,65fa81cf55f9b067e446b1a60510ebf2f449fa973105bd...,65fa81cf55f9b067e446b1a60510ebf2f449fa973105bd...,Lake Vanessaside,...,Car,413,Bachelor's,Student,3,1,0,Single,Rent,IT
9996,33a086bfa12208cd9e4557f8b9afb8074bd6cf9a858ce1...,33a086bfa12208cd9e4557f8b9afb8074bd6cf9a858ce1...,33a086bfa12208cd9e4557f8b9afb8074bd6cf9a858ce1...,Male,1978-02-11,47,33a086bfa12208cd9e4557f8b9afb8074bd6cf9a858ce1...,33a086bfa12208cd9e4557f8b9afb8074bd6cf9a858ce1...,33a086bfa12208cd9e4557f8b9afb8074bd6cf9a858ce1...,Charlesshire,...,Medical,761,Bachelor's,Student,27,5,4,Widowed,Mortgage,Construction
9997,ef8aa3ab5e36175b6392caaf9ae93c6e5e7844c1f3e52d...,ef8aa3ab5e36175b6392caaf9ae93c6e5e7844c1f3e52d...,ef8aa3ab5e36175b6392caaf9ae93c6e5e7844c1f3e52d...,Female,1999-08-31,26,ef8aa3ab5e36175b6392caaf9ae93c6e5e7844c1f3e52d...,ef8aa3ab5e36175b6392caaf9ae93c6e5e7844c1f3e52d...,ef8aa3ab5e36175b6392caaf9ae93c6e5e7844c1f3e52d...,Trevorville,...,Home,466,Associate Degree,Unemployed,0,1,0,Divorced,Own,Hospitality
9998,6d84c186b5299970171a11d169eb2f8bbbfb80dc90f146...,6d84c186b5299970171a11d169eb2f8bbbfb80dc90f146...,6d84c186b5299970171a11d169eb2f8bbbfb80dc90f146...,Male,1988-03-07,37,6d84c186b5299970171a11d169eb2f8bbbfb80dc90f146...,6d84c186b5299970171a11d169eb2f8bbbfb80dc90f146...,6d84c186b5299970171a11d169eb2f8bbbfb80dc90f146...,New Richard,...,Home,743,PhD,Self-employed,14,1,0,Single,Rent,Construction
